In [27]:
from transformers import AutoTokenizer, AutoModel
import torch

# 모델과 토크나이저 로드
model_name = "monologg/kobert"

model = AutoModel.from_pretrained("monologg/kobert")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)


print(f"모델 로드 완료: {model_name}")

모델 로드 완료: monologg/kobert


In [28]:
# 샘플 텍스트
text = "안녕하세요. 한국어 BERT 모델을 테스트합니다."

# 토큰화
tokens = tokenizer.tokenize(text)
print(f"토큰화 결과: {tokens}")

# 인코딩 (모델 입력용)
encoded = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

print(f"입력 ID 형태: {encoded['input_ids'].shape}")

토큰화 결과: ['▁안', '녕', '하세요', '.', '▁한국', '어', '▁B', 'ER', 'T', '▁모델', '을', '▁테스트', '합니다', '.']
입력 ID 형태: torch.Size([1, 128])


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

class MovieRecommendationSystem:
    def __init__(self):
        self.kobert_tokenizer = None
        self.kobert_model = None
        self.tfidf_title = None
        self.tfidf_genre = None
        self.mlb_genre = None
        self.movie_data = None
        self.plot_embeddings = None
        self.title_matrix = None
        self.genre_matrix = None
        
    def load_kobert_model(self):
        """KoBERT 모델 로드"""
        print("KoBERT 모델 로딩 중...")
        model_name = "monologg/kobert"
        self.kobert_model = AutoModel.from_pretrained(model_name)
        self.kobert_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.kobert_model.eval()
        print("KoBERT 모델 로드 완료")
    
    def get_kobert_embedding(self, text):
        """KoBERT를 사용하여 텍스트 임베딩 생성"""
        if not text or pd.isna(text):
            return np.zeros(768)  # KoBERT 임베딩 차원
        
        # 텍스트 길이 제한 (KoBERT 최대 입력 길이 고려)
        text = str(text)[:500]
        
        inputs = self.kobert_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )
        
        with torch.no_grad():
            outputs = self.kobert_model(**inputs)
            # [CLS] 토큰의 임베딩 사용
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().cpu().numpy()
        
        return embedding
    
    def prepare_data(self, movies_df):
        """데이터 전처리 및 특성 추출"""
        print("데이터 전처리 중...")
        
        # 필수 컬럼 확인
        required_columns = ['title', 'overview', 'genres']
        for col in required_columns:
            if col not in movies_df.columns:
                raise ValueError(f"필수 컬럼 '{col}'이 데이터에 없습니다.")
        
        # 결측값 처리
        movies_df['title'] = movies_df['title'].fillna('')
        movies_df['overview'] = movies_df['overview'].fillna('')
        movies_df['genres'] = movies_df['genres'].fillna('')
        
        self.movie_data = movies_df.copy()
        
        # 1. 제목 TF-IDF 벡터화
        print("제목 TF-IDF 벡터화 중...")
        self.tfidf_title = TfidfVectorizer(
            max_features=5000,
            stop_words=None,
            ngram_range=(1, 2),
            min_df=2
        )
        self.title_matrix = self.tfidf_title.fit_transform(movies_df['title'])
        
        # 2. 장르 처리 (One-hot 인코딩)
        print("장르 One-hot 인코딩 중...")
        # 장르가 문자열인 경우 리스트로 변환
        genre_lists = []
        for genres in movies_df['genres']:
            if isinstance(genres, str) and genres:
                # 장르가 콤마로 구분된 문자열인 경우
                genre_list = [g.strip() for g in genres.split(',')]
                genre_lists.append(genre_list)
            elif isinstance(genres, list):
                genre_lists.append(genres)
            else:
                genre_lists.append([])
        
        self.mlb_genre = MultiLabelBinarizer()
        self.genre_matrix = self.mlb_genre.fit_transform(genre_lists)
        
        # 3. 줄거리 KoBERT 임베딩
        print("줄거리 KoBERT 임베딩 생성 중...")
        if self.kobert_model is None:
            self.load_kobert_model()
        
        plot_embeddings = []
        for i, overview in enumerate(movies_df['overview']):
            if i % 100 == 0:
                print(f"줄거리 임베딩 진행률: {i}/{len(movies_df)}")
            embedding = self.get_kobert_embedding(overview)
            plot_embeddings.append(embedding)
        
        self.plot_embeddings = np.array(plot_embeddings)
        print("데이터 전처리 완료")
    
    def calculate_similarity(self, movie_idx, weights):
        """특정 영화와 다른 모든 영화 간의 유사도 계산"""
        # weights는 반드시 전달되어야 함 (recommend_movies에서 처리)
        
        # 1. 줄거리 유사도 (KoBERT 임베딩 기반)
        target_plot_embedding = self.plot_embeddings[movie_idx].reshape(1, -1)
        plot_similarities = cosine_similarity(target_plot_embedding, self.plot_embeddings)[0]
        
        # 2. 제목 유사도 (TF-IDF 기반)
        target_title_vector = self.title_matrix[movie_idx]
        title_similarities = cosine_similarity(target_title_vector, self.title_matrix)[0]
        
        # 3. 장르 유사도 (One-hot 인코딩 기반)
        target_genre_vector = self.genre_matrix[movie_idx].reshape(1, -1)
        genre_similarities = cosine_similarity(target_genre_vector, self.genre_matrix)[0]
        
        # 4. 가중 평균으로 최종 유사도 계산
        final_similarities = (
            weights['plot'] * plot_similarities +
            weights['title'] * title_similarities +
            weights['genre'] * genre_similarities
        )
        
        return final_similarities, plot_similarities, title_similarities, genre_similarities
    
    def recommend_movies(self, movie_title=None, movie_idx=None, top_n=10, weights=None):
        """영화 추천"""
        
        # 기본 가중치 설정
        if weights is None:
            weights = {'plot': 0.7, 'title': 0.15, 'genre': 0.15}
        
        if movie_idx is None:
            if movie_title is None:
                raise ValueError("movie_title 또는 movie_idx 중 하나는 제공되어야 합니다.")
            
            # 제목으로 인덱스 찾기
            matches = self.movie_data[self.movie_data['title'].str.contains(movie_title, case=False, na=False)]
            if matches.empty:
                print(f"'{movie_title}'과 일치하는 영화를 찾을 수 없습니다.")
                return None
            
            movie_idx = matches.index[0]
            actual_title = matches.iloc[0]['title']
        else:
            actual_title = self.movie_data.iloc[movie_idx]['title']
        
        print(f"\n기준 영화: {actual_title}")
        print(f"장르: {self.movie_data.iloc[movie_idx]['genres']}")
        print(f"줄거리: {self.movie_data.iloc[movie_idx]['overview'][:200]}...")
        print(f"사용된 가중치: {weights}")  # 가중치 출력 추가
        
        # 유사도 계산 (weights 매개변수 전달)
        final_similarities, plot_sim, title_sim, genre_sim = self.calculate_similarity(movie_idx, weights)
        
        # 자기 자신 제외하고 상위 N개 추천
        similar_indices = np.argsort(final_similarities)[::-1][1:top_n+1]
        
        # 추천 결과 생성
        recommendations = []
        for idx in similar_indices:
            movie_info = {
                'title': self.movie_data.iloc[idx]['title'],
                'genres': self.movie_data.iloc[idx]['genres'],
                'overview': self.movie_data.iloc[idx]['overview'],
                'total_similarity': final_similarities[idx],
                'plot_similarity': plot_sim[idx],
                'title_similarity': title_sim[idx],
                'genre_similarity': genre_sim[idx]
            }
            recommendations.append(movie_info)
        
        return recommendations
    
    def display_recommendations(self, recommendations):
        """추천 결과 출력"""
        if not recommendations:
            print("추천할 영화가 없습니다.")
            return
        
        print(f"\n=== 추천 영화 TOP {len(recommendations)} ===")
        
        for i, movie in enumerate(recommendations, 1):
            print(f"\n{i}. {movie['title']}")
            print(f"   장르: {movie['genres']}")
            print(f"   총 유사도: {movie['total_similarity']:.3f}")
            print(f"   세부 유사도 - 줄거리: {movie['plot_similarity']:.3f}, "
                  f"제목: {movie['title_similarity']:.3f}, "
                  f"장르: {movie['genre_similarity']:.3f}")
            print(f"   줄거리: {movie['overview'][:150]}...")



c:\Users\hyewo\Documents\ItemBasedCollaborative\kobert_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 사용 예시
def create_sample_data():
    """샘플 데이터 생성"""
    sample_movies = [
        {
            'title': '기생충',
            'overview': '전원 백수로 살 길 막막하지만 사이는 좋은 기택 가족. 장남 기우가 명문대생 친구의 소개로 박 사장 집 가정교사 일을 하게 되면서 운명이 바뀌기 시작한다.',
            'genres': '드라마, 스릴러, 코미디'
        },
        {
            'title': '아바타',
            'overview': '2154년, 지구의 에너지 고갈로 인해 판도라 행성으로 떠난 인간들이 그곳에서 벌이는 전쟁을 그린 SF 영화.',
            'genres': 'SF, 액션, 어드벤처'
        },
        {
            'title': '타이타닉',
            'overview': '1912년 침몰한 타이타닉호를 배경으로 한 로맨스 영화. 서로 다른 계층의 잭과 로즈의 사랑 이야기.',
            'genres': '로맨스, 드라마'
        },
        {
            'title': '어벤져스',
            'overview': '지구를 위협하는 적으로부터 세상을 구하기 위해 아이언맨, 토르, 캡틴 아메리카 등 슈퍼히어로들이 모인다.',
            'genres': '액션, SF, 어드벤처'
        },
        {
            'title': '부산행',
            'overview': '정체불명의 바이러스가 전국으로 확산되고, KTX를 타고 부산으로 향하는 사람들의 생존 이야기.',
            'genres': '액션, 호러, 스릴러'
        },
        {
            'title': '캡틴 아메리카: 시빌 워',
            'overview': '초인 등록제를 둘러싸고 슈퍼히어로들이 둘로 나뉘어 대립한다. 아이언맨과 캡틴 아메리카가 이끄는 연합의 충돌.',
            'genres': '액션, SF, 드라마'
        },
        {
            'title': '가디언즈 오브 갤럭시',
            'overview': '우주 최강의 악당으로부터 은하계를 지키기 위해 모인 개성 넘치는 히어로들의 이야기.',
            'genres': 'SF, 액션, 어드벤처'
        },
        {
            'title': '블랙 팬서',
            'overview': '와칸다 왕국의 왕이자 히어로 블랙 팬서가 나라를 지키고 지구의 운명을 건 전쟁에 나선다.',
            'genres': '액션, SF, 드라마'
        },
        {
            'title': '아바타: 물의 길',
            'overview': '판도라 행성의 바다 부족과 함께 가족을 지키기 위한 전쟁이 다시 시작된다.',
            'genres': 'SF, 어드벤처, 드라마'
        },
        {
            'title': '인터스텔라',
            'overview': '지구의 생존이 위협받는 가운데, 새로운 행성을 찾아 우주를 여행하는 인간들의 이야기.',
            'genres': 'SF, 드라마'
        },
        {
            'title': '가타카',
            'overview': '유전자 기반 사회에서 한 남자가 우주 비행사의 꿈을 이루기 위해 싸우는 미래 SF 영화.',
            'genres': 'SF, 드라마'
        }
    ]
    
    return pd.DataFrame(sample_movies)

# 실행 예시
if __name__ == "__main__":
    # 샘플 데이터 생성
    movies_df = create_sample_data()
    
    # 추천 시스템 초기화
    recommender = MovieRecommendationSystem()
    
    # 데이터 전처리
    recommender.prepare_data(movies_df)
    
    # # 영화 추천
    # recommendations = recommender.recommend_movies(
    #     movie_title='기생충',
    #     top_n=3,
    #     weights={'plot': 0.7, 'title': 0.15, 'genre': 0.15}
    # )
    
    # # 결과 출력
    # recommender.display_recommendations(recommendations)
    
    # print("\n" + "="*50)
    # print("다른 가중치로 추천해보기")
    # print("="*50)
    
    # 다른 가중치로 추천
    recommendations2 = recommender.recommend_movies(
        movie_title='어벤져스',
        top_n=3,
        weights={'plot': 0.7, 'title': 0.2, 'genre': 0.1}
    )
    
    recommender.display_recommendations(recommendations2)

데이터 전처리 중...
제목 TF-IDF 벡터화 중...
장르 One-hot 인코딩 중...
줄거리 KoBERT 임베딩 생성 중...
KoBERT 모델 로딩 중...


c:\Users\hyewo\Documents\ItemBasedCollaborative\kobert_env\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KoBERT 모델 로드 완료
줄거리 임베딩 진행률: 0/11
데이터 전처리 완료

기준 영화: 어벤져스
장르: 액션, SF, 어드벤처
줄거리: 지구를 위협하는 적으로부터 세상을 구하기 위해 아이언맨, 토르, 캡틴 아메리카 등 슈퍼히어로들이 모인다....
사용된 가중치: {'plot': 0.7, 'title': 0.2, 'genre': 0.1}

=== 추천 영화 TOP 3 ===

1. 가디언즈 오브 갤럭시
   장르: SF, 액션, 어드벤처
   총 유사도: 0.673
   세부 유사도 - 줄거리: 0.819, 제목: 0.000, 장르: 1.000
   줄거리: 우주 최강의 악당으로부터 은하계를 지키기 위해 모인 개성 넘치는 히어로들의 이야기....

2. 캡틴 아메리카: 시빌 워
   장르: 액션, SF, 드라마
   총 유사도: 0.642
   세부 유사도 - 줄거리: 0.822, 제목: 0.000, 장르: 0.667
   줄거리: 초인 등록제를 둘러싸고 슈퍼히어로들이 둘로 나뉘어 대립한다. 아이언맨과 캡틴 아메리카가 이끄는 연합의 충돌....

3. 아바타
   장르: SF, 액션, 어드벤처
   총 유사도: 0.638
   세부 유사도 - 줄거리: 0.768, 제목: 0.000, 장르: 1.000
   줄거리: 2154년, 지구의 에너지 고갈로 인해 판도라 행성으로 떠난 인간들이 그곳에서 벌이는 전쟁을 그린 SF 영화....


In [ ]:
   # def get_kobert_embedding(self, text):
    #     """KoBERT를 사용하여 텍스트 임베딩 생성"""
    #     if not text or pd.isna(text):
    #         return np.zeros(768)
        
    #     # 텍스트 전처리
    #     text = self.enhanced_text_preprocessing(text)
    #     text = text[:500]  # 길이 제한
        
    #     inputs = self.kobert_tokenizer(
    #         text,
    #         return_tensors="pt",
    #         truncation=True,
    #         padding=True,
    #         max_length=512
    #     )
        
    #     with torch.no_grad():
    #         outputs = self.kobert_model(**inputs)
            
    #         # [CLS] 토큰 임베딩
    #         cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().cpu().numpy()
            
    #         # 전체 토큰 평균 임베딩
    #         attention_mask = inputs['attention_mask']
    #         token_embeddings = outputs.last_hidden_state
            
    #         mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    #         masked_embeddings = token_embeddings * mask_expanded
    #         summed_embeddings = torch.sum(masked_embeddings, 1)
    #         summed_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    #         mean_embedding = (summed_embeddings / summed_mask).squeeze().detach().cpu().numpy()
            
    #         # 결합 임베딩
    #         combined_embedding = 0.7 * cls_embedding + 0.3 * mean_embedding
            
    #         # L2 정규화 추가
    #         norm = np.linalg.norm(combined_embedding)
    #         if norm > 0:
    #             combined_embedding = combined_embedding / norm
                
    #         return combined_embedding

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import re
import os
from collections import Counter

class ImprovedMovieRecommendationSystem:
    def __init__(self):
        self.kobert_tokenizer = None
        self.kobert_model = None
        self.tfidf_title = None
        self.tfidf_overview = None
        self.tfidf_keywords = None
        self.mlb_genre = None
        self.movie_data = None
        self.plot_embeddings = None
        self.title_matrix = None
        self.genre_matrix = None
        self.keyword_matrix = None
        self.overview_matrix = None
        
    def load_kobert_model(self):
        """KoBERT 모델 로드"""
        print("KoBERT 모델 로딩 중...")
        model_name = "monologg/kobert"
        self.kobert_model = AutoModel.from_pretrained(model_name)
        self.kobert_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.kobert_model.eval()
        print("KoBERT 모델 로드 완료")

    def extract_nouns_from_text(text, okt):
        """텍스트에서 명사 추출"""
        if not text:
            return []
        
        # 명사 추출
        nouns = okt.nouns(text)
        
        # 단어 길이가 1 이하인 것 제거
        filtered_nouns = [noun for noun in nouns if len(noun) > 1]
        
        return filtered_nouns
    
    def extract_keywords(self, text):
        """텍스트에서 키워드 추출"""
        if not text or pd.isna(text):
            return []
        
        # 한국어 영화 관련 키워드 패턴
        movie_keywords = {
             # 장르 관련 (확장)
            'action': [
                '액션', '전투', '싸움', '전쟁', '격투', '추격', '폭발', '총격',
                '무술', '카체이싱', '건파이트', '배틀', '액션스릴러', '느와르',
                '스파이', '첩보', '잠입', '미션', '작전', '복수', '정의'
            ],
            'romance': [
                '사랑', '로맨스', '연인', '결혼', '연애', '첫사랑', '이별', '만남',
                '멜로', '러브스토리', '운명', '재회', '프로포즈', '웨딩', '데이트',
                '썸', '고백', '짝사랑', '원거리', '국제연애', '나이차', '사내연애'
            ],
            'comedy': [
                '코미디', '웃음', '유머', '재미', '개그', '유쾌', '농담',
                '개그맨', '상황극', '슬랩스틱', '로맨틱코미디', '패밀리코미디',
                '블랙코미디', '풍자', '해학', '익살', '코믹', '유머러스'
            ],
            'horror': [
                '공포', '호러', '무서운', '귀신', '좀비', '괴물', '악령',
                '사이코', '살인마', '연쇄살인', '초자연', '오컬트', '엑소시즘',
                '저주', '원혼', '귀신', '유령', '무덤', '폐가', '심령'
            ],
            'thriller': [
                '스릴러', '긴장', '추적', '수사', '범죄', '살인', '미스터리',
                '서스펜스', '추리', '탐정', '형사', '검찰', '법정', '재판',
                '납치', '협박', '음모', '배신', '사기', '해킹', '첩보'
            ],
            'drama': [
                '드라마', '인간', '감동', '눈물', '가족', '인생', '성장',
                '휴먼드라마', '가족드라마', '성장드라마', '시대드라마',
                '사회드라마', '의료드라마', '법정드라마', '학원드라마'
            ],
            'SF': [
                'SF', '미래', '우주', '로봇', '과학', '기술', '외계', '시간여행',
                '사이버펑크', '디스토피아', '유토피아', 'AI', '인공지능',
                '가상현실', '바이오', '복제', '돌연변이', '포스트아포칼립스'
            ],
            'fantasy': [
                '판타지', '마법', '신화', '용', '마술', '초자연',
                '다크판타지', '어반판타지', '하이판타지', '무협', '환상',
                '마법사', '마녀', '요정', '전설', '신화'
            ],
            
            # 세부 장르 추가
            'documentary': [
                '다큐멘터리', '다큐', '실화', '실제', '기록', '취재',
                '사회고발', '환경', '역사', '인물', '음악다큐', '여행다큐'
            ],
            'animation': [
                '애니메이션', '애니', '만화', '3D', '2D', '스톱모션',
                '가족애니', '성인애니', 'CGI', '픽사', '디즈니'
            ],
            'musical': [
                '뮤지컬', '음악', '노래', '춤', '공연', '오페라',
                '밴드', '가수', '댄스', '음악영화', '콘서트'
            ],
            'western': [
                '서부', '카우보이', '총잡이', '보안관', '무법자',
                '황야', '술집', '결투', '정착민'
            ],
            'sports': [
                '스포츠', '운동', '축구', '야구', '농구', '복싱',
                '마라톤', '올림픽', '경기', '팀', '코치', '선수'
            ],
            
            # 인물 관련 (확장)
            'hero': [
                '히어로', '영웅', '주인공', '구원', '슈퍼히어로',
                '구세주', '리더', '대장', '캡틴', '왕', '황제', '장군'
            ],
            'villain': [
                '악역', '악당', '범죄자', '적', '보스',
                '마피아', '조직', '테러리스트', '독재자', '사이코패스'
            ],
            'family': [
                '아버지', '어머니', '아들', '딸', '형제', '자매',
                '할아버지', '할머니', '삼촌', '이모', '고모', '숙부',
                '사위', '며느리', '손자', '손녀', '조카', '입양'
            ],
            'profession': [
                '의사', '변호사', '교사', '경찰', '소방관', '군인',
                '기자', '작가', '화가', '음악가', '요리사', '사업가',
                '정치인', '연예인', '운동선수', '과학자', '엔지니어'
            ],
            
            # 배경 관련 (확장)
            'modern': [
                '현대', '도시', '서울', '뉴욕', '일상',
                '아파트', '오피스', '카페', '클럽', '쇼핑몰',
                '지하철', '고속도로', '공항', '호텔'
            ],
            'historical': [
                '역사', '전통', '고대', '중세', '조선',
                '임진왜란', '일제강점기', '6.25', '80년대', '90년대',
                '궁궐', '한옥', '전쟁', '왕조', '신라', '고구려', '백제'
            ],
            'future': [
                '미래', '2030', '2050', '디스토피아',
                '포스트아포칼립스', '사이버펑크', '스페이스오페라'
            ],
            'space': [
                '우주', '행성', '은하', '우주선', '외계',
                '화성', '달', '소행성', '블랙홀', '워프'
            ],
            'school': [
                '학교', '대학', '학생', '교실', '캠퍼스',
                '초등학교', '중학교', '고등학교', '대학교', '대학원',
                '기숙사', '동아리', '학회', '졸업', '입학'
            ],
            'workplace': [
                '회사', '직장', '사무실', '공장', '병원',
                '법원', '경찰서', '방송국', '신문사', '은행'
            ],
            
            # 감정/분위기 관련
            'emotion': [
                '감동', '눈물', '슬픔', '기쁨', '행복', '분노',
                '절망', '희망', '사랑', '이별', '그리움', '향수',
                '외로움', '고독', '우울', '스트레스'
            ],
            'mood': [
                '다크', '밝은', '유쾌', '우울', '무거운', '가벼운',
                '긴장감', '서정적', '잔잔한', '격렬한', '평화로운'
            ],
            
            # 소재/테마 관련
            'theme': [
                '복수', '정의', '우정', '배신', '희생',
                '성장', '자아실현', '꿈', '도전', '극복', '화해',
                '갈등', '대립', '협력', '경쟁', '성공', '실패'
            ],
            'social_issue': [
                '사회문제', '부패', '불평등', '차별', '환경',
                '정치', '경제', '교육', '의료', '복지',
                '성차별', '연령차별', '계층갈등', '지역갈등'
            ]
        }
        
        # 키워드 추출
        extracted_keywords = []
        text_lower = text.lower()
        
        for category, keywords in movie_keywords.items():
            for keyword in keywords:
                if keyword in text_lower:
                    extracted_keywords.append(keyword)
        
        return extracted_keywords



    def get_kobert_embedding(self, text):
        """KoBERT를 사용하여 텍스트 임베딩 생성"""
        if not text or pd.isna(text):
            return np.zeros(768)  # KoBERT 임베딩 차원
        
        text = self.enhanced_text_preprocessing(text)
        
        # 텍스트 길이 제한 (KoBERT 최대 입력 길이 고려)
        text = str(text)[:500]
        
        inputs = self.kobert_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )
        
        with torch.no_grad():
            outputs = self.kobert_model(**inputs)
            # [CLS] 토큰의 임베딩 사용
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().cpu().numpy()
        
        return embedding
    
    def calculate_genre_similarity_advanced(self, target_genres, candidate_genres):
        """고급 장르 유사도 계산"""
        # 장르 간 유사도 매트릭스 (실제로는 더 정교하게 구성 가능)
        genre_similarity_matrix = {
            '액션': {
                '모험': 0.8,
                'SF': 0.7,
                '스릴러': 0.6,
                '범죄': 0.5,
                '판타지': 0.4,
                '전쟁': 0.6,
                '서부': 0.5
            },
            
            '모험': {
                '액션': 0.8,
                '판타지': 0.7,
                'SF': 0.6,
                '가족': 0.6,
                '애니메이션': 0.5,
                '역사': 0.4
            },
            
            '애니메이션': {
                '가족': 0.8,
                '코미디': 0.6,
                '판타지': 0.5,
                '모험': 0.5,
                '로맨스': 0.4,
                '드라마': 0.3
            },
            
            '코미디': {
                '로맨스': 0.5,
                '드라마': 0.4,
                '애니메이션': 0.6,
                '가족': 0.5,
                '음악': 0.3
            },
            
            '범죄': {
                '스릴러': 0.8,
                '미스터리': 0.7,
                '드라마': 0.6,
                '액션': 0.5,
                '공포': 0.4
            },
            
            '다큐멘터리': {
                '역사': 0.6,
                '드라마': 0.4,
                '음악': 0.3,
                '전쟁': 0.3
            },
            
            '드라마': {
                '로맨스': 0.7,
                '스릴러': 0.5,
                '코미디': 0.4,
                '범죄': 0.6,
                '미스터리': 0.5,
                '역사': 0.4,
                '다큐멘터리': 0.4,
                '가족': 0.5,
                '음악': 0.4
            },
            
            '가족': {
                '애니메이션': 0.8,
                '코미디': 0.5,
                '모험': 0.6,
                '판타지': 0.5,
                '드라마': 0.5,
                '로맨스': 0.4
            },
            
            '판타지': {
                '모험': 0.7,
                'SF': 0.6,
                '액션': 0.4,
                '애니메이션': 0.5,
                '가족': 0.5,
                '공포': 0.5
            },
            
            '역사': {
                '드라마': 0.4,
                '전쟁': 0.8,
                '다큐멘터리': 0.6,
                '모험': 0.4,
                '서부': 0.5
            },
            
            '공포': {
                '스릴러': 0.7,
                '미스터리': 0.6,
                '판타지': 0.5,
                'SF': 0.3,
                '범죄': 0.4
            },
            
            '음악': {
                '드라마': 0.4,
                '코미디': 0.3,
                '로맨스': 0.5,
                '다큐멘터리': 0.3
            },
            
            '미스터리': {
                '범죄': 0.7,
                '스릴러': 0.8,
                '공포': 0.6,
                '드라마': 0.5
            },
            
            '로맨스': {
                '드라마': 0.7,
                '코미디': 0.5,
                '음악': 0.5,
                '애니메이션': 0.4,
                '가족': 0.4
            },
            
            'SF': {
                '액션': 0.7,
                '모험': 0.6,
                '판타지': 0.6,
                '스릴러': 0.5,
                '공포': 0.3
            },
            
            '스릴러': {
                '범죄': 0.8,
                '미스터리': 0.8,
                '공포': 0.7,
                '액션': 0.6,
                '드라마': 0.5,
                'SF': 0.5
            },
            
            '전쟁': {
                '역사': 0.8,
                '드라마': 0.6,
                '액션': 0.6,
                '다큐멘터리': 0.3
            },
            
            '서부': {
                '액션': 0.5,
                '역사': 0.5,
                '드라마': 0.4
            }
        }
        
        if not target_genres or not candidate_genres:
            return 0.0
        
        # 직접 매치 점수
        direct_match = len(set(target_genres) & set(candidate_genres)) / len(set(target_genres) | set(candidate_genres))
        
        # 간접 매치 점수
        indirect_score = 0
        for target_genre in target_genres:
            for candidate_genre in candidate_genres:
                if target_genre.lower() in genre_similarity_matrix:
                    if candidate_genre.lower() in genre_similarity_matrix[target_genre.lower()]:
                        indirect_score += genre_similarity_matrix[target_genre.lower()][candidate_genre.lower()]
        
        indirect_score = indirect_score / (len(target_genres) * len(candidate_genres)) if target_genres and candidate_genres else 0
        
        return 0.7 * direct_match + 0.3 * indirect_score
    
    def prepare_data(self, movies_df, force_recompute=False):
        """데이터 전처리 및 특성 추출"""
        print("데이터 전처리 중...")
        
        # 필수 컬럼 확인
        required_columns = ['title', 'overview', 'genres']
        for col in required_columns:
            if col not in movies_df.columns:
                raise ValueError(f"필수 컬럼 '{col}'이 데이터에 없습니다.")
        
        # 결측값 처리
        movies_df['title'] = movies_df['title'].fillna('')
        movies_df['overview'] = movies_df['overview'].fillna('')
        movies_df['genres'] = movies_df['genres'].fillna('')
        
        # 줄거리 텍스트 정제
        movies_df['overview_clean'] = movies_df['overview'].apply(self.enhanced_text_preprocessing)
        
        # 키워드 추출
        print("키워드 추출 중...")
        movies_df['keywords'] = movies_df['overview'].apply(self.extract_keywords)
        movies_df['keywords_text'] = movies_df['keywords'].apply(lambda x: ' '.join(x) if x else '')
        
        self.movie_data = movies_df.copy()
        
        # 1. 제목 TF-IDF 벡터화
        print("제목 TF-IDF 벡터화 중...")
        self.tfidf_title = TfidfVectorizer(
            max_features=5000,
            stop_words=None,
            ngram_range=(1, 2),
            min_df=1
        )
        self.title_matrix = self.tfidf_title.fit_transform(movies_df['title'])
        
        # 2. 줄거리 TF-IDF 벡터화 (보조 특성)
        print("줄거리 TF-IDF 벡터화 중...")
        self.tfidf_overview = TfidfVectorizer(
            max_features=10000,
            stop_words=None,
            ngram_range=(1, 2),
            min_df=1
        )
        overview_tfidf_matrix = self.tfidf_overview.fit_transform(movies_df['overview'])

        # LSA (Truncated SVD)
        self.svd = TruncatedSVD(n_components=100, random_state=42)
        self.overview_matrix = self.svd.fit_transform(overview_tfidf_matrix)
        
        # 3. 키워드 TF-IDF 벡터화
        print("키워드 TF-IDF 벡터화 중...")
        self.tfidf_keywords = TfidfVectorizer(
            max_features=1000,
            stop_words=None,
            ngram_range=(1, 2),
            min_df=1
        )
        self.keyword_matrix = self.tfidf_keywords.fit_transform(movies_df['keywords_text'])
        
        # 4. 장르 처리
        print("장르 처리 중...")
        genre_lists = []
        for genres in movies_df['genres']:
            if isinstance(genres, str) and genres:
                genre_list = [g.strip() for g in genres.split(',')]
                genre_lists.append(genre_list)
            elif isinstance(genres, list):
                genre_lists.append(genres)
            else:
                genre_lists.append([])
        
        self.mlb_genre = MultiLabelBinarizer()
        self.genre_matrix = self.mlb_genre.fit_transform(genre_lists)

        # 5. KoBERT 임베딩
        if not force_recompute and os.path.exists("plot_embeddings.npy"):
            print("✅ 저장된 줄거리 임베딩을 불러옵니다.")
            self.plot_embeddings = np.load("plot_embeddings.npy")
        else:
            print("🛠 줄거리 임베딩을 새로 생성합니다.")
            if self.kobert_model is None:
                self.load_kobert_model()
        
            plot_embeddings = []
            for i, overview in enumerate(movies_df['overview']):
                if i % 50 == 0:
                    print(f"임베딩 진행률: {i}/{len(movies_df)}")
                embedding = self.get_kobert_embedding(overview)
                plot_embeddings.append(embedding)

            self.plot_embeddings = np.array(plot_embeddings)
            np.save("plot_embeddings.npy", self.plot_embeddings)
            print("줄거리 임베딩 저장 완료: plot_embeddings.npy")
            print("데이터 전처리 완료")
    
    def calculate_similarity_comprehensive(self, movie_idx, weights):
        """포괄적인 유사도 계산"""
        
        # 1. KoBERT 임베딩 기반 유사도 (의미적 유사도)
        target_plot_embedding = self.plot_embeddings[movie_idx].reshape(1, -1)
        kobert_similarities = cosine_similarity(target_plot_embedding, self.plot_embeddings)[0]
        
        # 2. 줄거리 TF-IDF 기반 유사도 (어휘적 유사도)
        target_overview_vector = self.overview_matrix[movie_idx]
        overview_similarities = cosine_similarity(target_overview_vector.reshape(1, -1), self.overview_matrix)[0]
        
        # 3. 제목 유사도
        target_title_vector = self.title_matrix[movie_idx]
        title_similarities = cosine_similarity(target_title_vector, self.title_matrix)[0]
        
        # 4. 키워드 유사도
        target_keyword_vector = self.keyword_matrix[movie_idx]
        keyword_similarities = cosine_similarity(target_keyword_vector, self.keyword_matrix)[0]
        
        # 5. 고급 장르 유사도
        target_genres = self.movie_data.iloc[movie_idx]['genres'].split(',') if self.movie_data.iloc[movie_idx]['genres'] else []
        target_genres = [g.strip() for g in target_genres]
        
        genre_similarities = []
        for i in range(len(self.movie_data)):
            candidate_genres = self.movie_data.iloc[i]['genres'].split(',') if self.movie_data.iloc[i]['genres'] else []
            candidate_genres = [g.strip() for g in candidate_genres]
            
            similarity = self.calculate_genre_similarity_advanced(target_genres, candidate_genres)
            genre_similarities.append(similarity)
        
        genre_similarities = np.array(genre_similarities)
        
        # 6. 가중 평균으로 최종 유사도 계산
        final_similarities = (
            weights['kobert'] * kobert_similarities +
            weights['overview_tfidf'] * overview_similarities +
            weights['title'] * title_similarities +
            weights['keywords'] * keyword_similarities +
            weights['genre'] * genre_similarities
        )
        
        return (final_similarities, kobert_similarities, overview_similarities, 
                title_similarities, keyword_similarities, genre_similarities)
    
    def adaptive_weights(self, movie_idx):
        """영화 특성에 따른 적응적 가중치 계산"""
        movie_info = self.movie_data.iloc[movie_idx]
        
        # 기본 가중치
        weights = {
            'kobert': 0.25,
            'overview_tfidf': 0.25,
            'title': 0.15,
            'keywords': 0.20,
            'genre': 0.15
        }
        
        # 영화 특성에 따른 가중치 조정
        
        # 1. 줄거리 길이에 따른 조정
        overview_length = len(movie_info['overview']) if movie_info['overview'] else 0
        if overview_length > 300:  # 긴 줄거리
            weights['kobert'] += 0.03
            weights['overview_tfidf'] += 0.03
            weights['title'] -= 0.03
            weights['keywords'] -= 0.03
        elif overview_length < 50:  # 짧은 줄거리
            weights['title'] += 0.10
            weights['genre'] += 0.10
            weights['kobert'] -= 0.10
            weights['overview_tfidf'] -= 0.10
        
        # 2. 장르 수에 따른 조정
        genre_count = len(movie_info['genres'].split(',')) if movie_info['genres'] else 0
        if genre_count > 3:  # 많은 장르
            weights['genre'] += 0.05
            weights['kobert'] -= 0.05
        elif genre_count == 1:  # 단일 장르
            weights['genre'] += 0.10
            weights['overview_tfidf'] -= 0.05
            weights['keywords'] -= 0.05
        
        # 3. 특정 장르에 대한 조정
        genres = movie_info['genres'].lower() if movie_info['genres'] else ''
        if 'sf' in genres or '액션' in genres:
            weights['keywords'] += 0.05
            weights['title'] -= 0.05
        elif '로맨스' in genres or '드라마' in genres:
            weights['kobert'] += 0.05
            weights['overview_tfidf'] += 0.05
            weights['keywords'] -= 0.05
            weights['genre'] -= 0.05
        
        # 가중치 정규화
        total_weight = sum(weights.values())
        weights = {k: v/total_weight for k, v in weights.items()}
        
        return weights
    
    def recommend_movies(self, movie_title=None, movie_idx=None, top_n=10, weights=None, use_adaptive_weights=True):
        """영화 추천"""
        
        if movie_idx is None:
            if movie_title is None:
                raise ValueError("movie_title 또는 movie_idx 중 하나는 제공되어야 합니다.")
            
            # 제목으로 인덱스 찾기
            matches = self.movie_data[self.movie_data['title'].str.contains(movie_title, case=False, na=False)]
            if matches.empty:
                print(f"'{movie_title}'과 일치하는 영화를 찾을 수 없습니다.")
                return None
            
            movie_idx = matches.index[0]
            actual_title = matches.iloc[0]['title']
        else:
            actual_title = self.movie_data.iloc[movie_idx]['title']
        
        # 가중치 결정
        if use_adaptive_weights:
            weights = self.adaptive_weights(movie_idx)
            print(f"적응적 가중치 사용: {weights}")
        elif weights is None:
            weights = {
                'kobert': 0.15,
                'overview_tfidf': 0.35,
                'title': 0.10,
                'keywords': 0.25,
                'genre': 0.15
            }
        
        print(f"\n기준 영화: {actual_title}")
        print(f"장르: {self.movie_data.iloc[movie_idx]['genres']}")
        print(f"줄거리: {self.movie_data.iloc[movie_idx]['overview'][:200]}...")
        
        # 유사도 계산
        similarities = self.calculate_similarity_comprehensive(movie_idx, weights)
        final_similarities = similarities[0]
        
        # 자기 자신 제외하고 상위 N개 추천
        similar_indices = np.argsort(final_similarities)[::-1][1:top_n+1]
        
        # 추천 결과 생성
        recommendations = []
        for idx in similar_indices:
            movie_info = {
                'title': self.movie_data.iloc[idx]['title'],
                'genres': self.movie_data.iloc[idx]['genres'],
                'overview': self.movie_data.iloc[idx]['overview'],
                'keywords': self.movie_data.iloc[idx]['keywords'],
                'total_similarity': final_similarities[idx],
                'kobert_similarity': similarities[1][idx],
                'overview_tfidf_similarity': similarities[2][idx],
                'title_similarity': similarities[3][idx],
                'keyword_similarity': similarities[4][idx],
                'genre_similarity': similarities[5][idx]
            }
            recommendations.append(movie_info)
        
        return recommendations
    
    def display_recommendations(self, recommendations):
        """추천 결과 출력"""
        if not recommendations:
            print("추천할 영화가 없습니다.")
            return
        
        print(f"\n=== 추천 영화 TOP {len(recommendations)} ===")
        
        for i, movie in enumerate(recommendations, 1):
            print(f"\n{i}. {movie['title']}")
            print(f"   장르: {movie['genres']}")
            print(f"   총 유사도: {movie['total_similarity']:.3f}")
            print(f"   세부 유사도:")
            print(f"     - KoBERT: {movie['kobert_similarity']:.3f}")
            print(f"     - 줄거리 TF-IDF: {movie['overview_tfidf_similarity']:.3f}")
            print(f"     - 제목: {movie['title_similarity']:.3f}")
            print(f"     - 키워드: {movie['keyword_similarity']:.3f}")
            print(f"     - 장르: {movie['genre_similarity']:.3f}")
            print(f"   키워드: {movie['keywords']}")
            print(f"   줄거리: {movie['overview'][:100]}...")

c:\Users\hyewo\Documents\ItemBasedCollaborative\kobert_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# 사용 예시

def create_extended_sample_data():
    # movie_data.csv 파일을 불러와서 DataFrame 반환
    df = pd.read_csv('./tranning-data/movie_data.csv', encoding='utf-8-sig')
    return df

# 실행 예시
if __name__ == "__main__":
    # 확장된 샘플 데이터 생성
    movies_df = create_extended_sample_data()
    
    # 개선된 추천 시스템 초기화
    recommender = ImprovedMovieRecommendationSystem()
    
    # 데이터 전처리
    recommender.prepare_data(movies_df)
    
    # 적응적 가중치로 추천
    print("=== 8월의 크리스마스 기반 추천 (적응적 가중치) ===")
    recommendations1 = recommender.recommend_movies(
        movie_title='8월의 크리스마스',
        top_n=10,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations1)
    
    # 다른 영화로 테스트
    print("\n=== 아바타 기반 추천 (적응적 가중치) ===")
    recommendations2 = recommender.recommend_movies(
        movie_title='아바타',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations2)
    
    # 로맨스 영화로 테스트
    print("\n=== 해리 포터와 죽음의 성물 2 기반 추천 (적응적 가중치) ===")
    recommendations3 = recommender.recommend_movies(
        movie_title='해리 포터와 죽음의 성물 2',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations3)

데이터 전처리 중...
키워드 추출 중...
제목 TF-IDF 벡터화 중...
줄거리 TF-IDF 벡터화 중...
키워드 TF-IDF 벡터화 중...
장르 처리 중...
✅ 저장된 줄거리 임베딩을 불러옵니다.
=== 8월의 크리스마스 기반 추천 (적응적 가중치) ===
적응적 가중치 사용: {'kobert': 0.3, 'overview_tfidf': 0.3, 'title': 0.15, 'keywords': 0.15000000000000002, 'genre': 0.09999999999999999}

기준 영화: 8월의 크리스마스
장르: 드라마, 로맨스
줄거리: "좋아하는 남자 친구 없어요?" 변두리 사진관에서 아버지를 모시고 사는 노총각 ‘정원’. 시한부 인생을 받아들이고 가족, 친구들과 담담한 이별을 준비하던 어느 날, 주차단속요원 '다림'을 만나게 되고 차츰 평온했던 일상이 흔들리기 시작한다. "아저씨, 왜 나만 보면 웃어요?" 밝고 씩씩하지만 무료한 일상에 지쳐가던 스무 살 주차단속요원 '다림'. 단속차량 ...

=== 추천 영화 TOP 10 ===

1. 나를 잊지 말아요
   장르: 드라마, 로맨스, 미스터리
   총 유사도: 0.441
   세부 유사도:
     - KoBERT: 0.891
     - 줄거리 TF-IDF: 0.334
     - 제목: 0.000
     - 키워드: 0.114
     - 장르: 0.562
   키워드: ['사랑', '눈물', '가족', '신', '가족', '친구', '병원', '눈물', '행복', '사랑']
   줄거리: 교통사고 후, 지난 10년의 기억이 지워진 남자 석원. 친구, 가족, 심지어 본인이 어떤 사람인지조차 흐릿해진 석원은 병원에서 우연히 자신을 보며 눈물을 흘리는 낯선 여자 진영을 ...

2. 화려한 휴가
   장르: 드라마, 역사
   총 유사도: 0.431
   세부 유사도:
     - KoBERT: 0.872
     - 줄거리 TF-IDF: 0.241
     - 제목: 0.000
 

In [ ]:
# 키워드 추천 개선
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import re
import os
from collections import Counter
from konlpy.tag import Okt

class ImprovedMovieRecommendationSystem:
    def __init__(self, stopword_files = None, cache_dir="./cached_features"):
        self.kobert_tokenizer = None
        self.kobert_model = None
        self.tfidf_title = None
        self.tfidf_overview = None
        self.tfidf_keywords = None
        self.mlb_genre = None
        self.movie_data = None
        self.plot_embeddings = None
        self.title_matrix = None
        self.genre_matrix = None
        self.keyword_matrix = None
        self.overview_matrix = None

        # 불용어 파일 리스트가 제공되면 로드, 아니면 기본 빈 set
        if stopword_files is None:
            # 기본 불용어 파일 (예시: 프로젝트 내에 git_stopwords.txt 하나만 있는 경우)
            # 또는 비어있는 set으로 시작하여 불용어 처리를 하지 않을 수도 있습니다.
            stopword_files = ["git_stopwords.txt"] # 기본 파일 경로 지정
        
        self.custom_stopwords = self.load_stopwords_from_files(stopword_files)
        
    def load_kobert_model(self):
        """KoBERT 모델 로드"""
        print("KoBERT 모델 로딩 중...")
        model_name = "monologg/kobert"
        self.kobert_model = AutoModel.from_pretrained(model_name)
        self.kobert_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.kobert_model.eval()
        print("KoBERT 모델 로드 완료")

    def enhanced_text_preprocessing(self, text):
        """텍스트 전처리 강화"""
        if not text or pd.isna(text):
            return ""
        
        text = str(text)
        
        # 특수 문자 및 숫자 처리
        text = re.sub(r'[^\w\s가-힣a-zA-Z0-9]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()

    def extract_nouns_from_text(self, text, okt):
        """텍스트에서 명사 추출"""
        if not text:
            return []
        
        # 명사 추출
        nouns = okt.nouns(text)
        
        # 단어 길이가 1 이하인 것 제거
        filtered_nouns = [noun for noun in nouns if len(noun) > 1]
        
        return filtered_nouns
    
    def load_stopwords_from_files(self, stopword_files):
        """여러 불용어 파일을 로드하여 하나의 set으로 통합"""
        combined_stopwords = set()
        
        base_dir = "./stopwords/"

        for file_name in stopword_files:
            file_path = os.path.join(base_dir, file_name)

            if os.path.exists(file_path):
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        words = [line.strip() for line in f if line.strip()]
                        combined_stopwords.update(words)
                    print(f"불용어 파일 로드 완료: {file_path} ({len(words)}개)")
                except Exception as e:
                    print(f"불용어 파일 로드 실패: {file_path} - {e}")
    
        if not combined_stopwords: # 로드된 불용어가 없을 경우 (파일이 없거나 비어있을 때)
            print("❗ 모든 불용어 파일을 로드할 수 없거나 파일이 비어 있어 기본 불용어를 사용합니다.")
            # 예비 불용어 리스트를 set으로 변환
            return set(['하다', '있다', '없다', '되다', '이다', '것', '수', '점', '말', '안', '때', '등', '통해'])
        
        return combined_stopwords

    def enhanced_text_preprocessing_kobert(self, text):
        if pd.isna(text) or not text:
            return ""

        text = self.enhanced_text_preprocessing(text)
        stopwords_pos = ['Josa', 'Eomi', 'Suffix', 'Punctuation'] 
        
        tokens = self.okt.pos(text, norm=True, stem=True)
        
        filtered_tokens = []
        for word, pos in tokens:
            if pos not in stopwords_pos:
                # 함수 내부에서 로드한 custom_stopwords 사용
                if word not in self.custom_stopwords: 
                    filtered_tokens.append(word)
        
        return ' '.join(filtered_tokens)
    
    def extract_keywords(self, text):
        """텍스트에서 키워드 추출 (형태소 분석 + 불용어 제거 + 가중치 적용)"""
        if not text or pd.isna(text):
            return []
        
        # OKT 형태소 분석기 초기화 (클래스 변수로 관리하면 더 효율적)
        if not hasattr(self, 'okt'):
            self.okt = Okt()
        
        # 불용어 로드 (처음 한 번만 실행)
        if not hasattr(self, 'stopwords'):
            stopword_files = [
                'common_name_stopwords.txt',
                'korean_adverb_stopwords.txt', 
                'general_stopwords.txt',
                'git_stopwords.txt'
            ]
            self.stopwords = self.load_stopwords_from_files(stopword_files)
            print(f"전체 불용어 개수: {len(self.stopwords)}")
        
        # 한국어 영화 관련 키워드 패턴 (가중치 포함)
        movie_keywords = {
            # 장르 관련 (가중치 2)
            'action': {
                'weight': 2,
                'keywords': [
                    '액션', '전투', '싸움', '전쟁', '격투', '추격', '폭발', '총격',
                    '무술', '카체이싱', '건파이트', '배틀', '액션스릴러', '느와르',
                    '스파이', '첩보', '잠입', '미션', '작전', '복수', '정의', '위장'
                ]
            },
            'romance': {
                'weight': 2,
                'keywords': [
                    '사랑', '로맨스', '연인', '결혼', '연애', '첫사랑', '이별', '만남',
                    '멜로', '러브스토리', '운명', '재회', '프로포즈', '웨딩', '데이트',
                    '썸', '고백', '짝사랑', '원거리', '국제연애', '나이차', '사내연애'
                ]
            },
            'comedy': {
                'weight': 2,
                'keywords': [
                    '코미디', '웃음', '유머', '재미', '개그', '유쾌', '농담',
                    '개그맨', '상황극', '슬랩스틱', '로맨틱코미디', '패밀리코미디',
                    '블랙코미디', '풍자', '해학', '익살', '코믹', '유머러스'
                ]
            },
            'horror': {
                'weight': 2,
                'keywords': [
                    '공포', '호러', '무서운', '귀신', '좀비', '괴물', '악령',
                    '사이코', '살인마', '연쇄살인', '초자연', '오컬트', '엑소시즘',
                    '저주', '원혼', '귀신', '유령', '무덤', '폐가', '심령', '위험구역'
                ]
            },
            'thriller': {
                'weight': 2,
                'keywords': [
                    '스릴러', '긴장', '추적', '수사', '범죄', '살인', '미스터리',
                    '서스펜스', '추리', '탐정', '형사', '검찰', '법정', '재판',
                    '납치', '협박', '음모', '배신', '사기', '해킹', '첩보'
                ]
            },
            'drama': {
                'weight': 2,
                'keywords': [
                    '인간', '감동', '눈물', '인생', '성장'
                ]
            },
            
            # 테마/소재 관련 (가중치 1.5)
            'theme': {
                'weight': 1.5,
                'keywords': [
                    '복수', '정의', '우정', '배신', '희생',
                    '성장', '자아실현', '꿈', '도전', '극복', '화해',
                    '갈등', '대립', '협력', '경쟁', '성공', '실패'
                ]
            },
            'emotion': {
                'weight': 1.5,
                'keywords': [
                    '감동', '눈물', '슬픔', '기쁨', '행복', '분노',
                    '절망', '희망', '사랑', '이별', '그리움', '향수',
                    '외로움', '고독', '우울', '스트레스', '끔찍한'
                ]
            },
            
            # 배경/직업 관련 (가중치 1)
            'profession': {
                'weight': 1,
                'keywords': [
                    '의사', '변호사', '교사', '경찰', '소방관', '군인',
                    '기자', '작가', '화가', '음악가', '요리사', '사업가',
                    '정치인', '연예인', '운동선수', '과학자', '엔지니어'
                ]
            },
            'background': {
                'weight': 1,
                'keywords': [
                    '현대', '도시', '서울', '학교', '대학', '회사', '직장',
                    '병원', '법원', '경찰서', '아파트', '카페'
                ]
            }
        }
        
        # 1. 텍스트 전처리
        preprocessed_text = self.enhanced_text_preprocessing(text)
        
        # 2. 형태소 분석으로 명사 추출
        nouns = self.extract_nouns_from_text(preprocessed_text, self.okt)
        
        # 3. 불용어 제거
        filtered_nouns = [noun for noun in nouns 
                        if noun not in self.stopwords]
        
        # 4. 명사 빈도 계산
        noun_counts = Counter(filtered_nouns)
        
        # 5. 영화 키워드 매칭 및 가중치 적용
        keyword_scores = {}
        
        for category, category_info in movie_keywords.items():
            weight = category_info['weight']
            keywords = category_info['keywords']
            
            for keyword in keywords:
                # 키워드가 추출된 명사에 있는지 확인
                if keyword in filtered_nouns:
                    score = noun_counts[keyword] * weight
                    if keyword in keyword_scores:
                        keyword_scores[keyword] += score
                    else:
                        keyword_scores[keyword] = score
        
        # 6. 점수 기준으로 정렬하여 상위 키워드 반환
        sorted_keywords = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)
        
        # 상위 20개 키워드만 반환 (키워드만, 점수 제외)
        top_keywords = [keyword for keyword, score in sorted_keywords[:20]]
        
        # 매칭되지 않은 중요 명사도 일부 포함 (빈도수 기준)
        remaining_nouns = [noun for noun in noun_counts.most_common(10) 
                        if noun[0] not in top_keywords]
        
        # 최종 키워드 결합
        final_keywords = top_keywords + [noun[0] for noun in remaining_nouns[:5]]
        
        return final_keywords[:25]  # 최대 25개 키워드 반환


    def get_kobert_embedding(self, text):
        """KoBERT를 사용하여 텍스트 임베딩 생성"""
        if not text or pd.isna(text):
            return np.zeros(768)  # KoBERT 임베딩 차원
        
        text = self.enhanced_text_preprocessing(text)
        
        # 텍스트 길이 제한 (KoBERT 최대 입력 길이 고려)
        text = str(text)[:512]
        
        inputs = self.kobert_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )
        
        with torch.no_grad():
            outputs = self.kobert_model(**inputs)
            # [CLS] 토큰의 임베딩 사용
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().cpu().numpy()
        
        return embedding
    
    def calculate_genre_similarity_advanced(self, target_genres, candidate_genres):
        """고급 장르 유사도 계산"""
        # 장르 간 유사도 매트릭스 (실제로는 더 정교하게 구성 가능)
        genre_similarity_matrix = {
            '액션': {
                '모험': 0.8,
                'SF': 0.7,
                '스릴러': 0.6,
                '범죄': 0.5,
                '판타지': 0.4,
                '전쟁': 0.6,
                '서부': 0.5
            },
            
            '모험': {
                '액션': 0.8,
                '판타지': 0.7,
                'SF': 0.6,
                '가족': 0.6,
                '애니메이션': 0.5,
                '역사': 0.4
            },
            
            '애니메이션': {
                '가족': 0.8,
                '코미디': 0.6,
                '판타지': 0.5,
                '모험': 0.5,
                '로맨스': 0.4,
                '드라마': 0.3
            },
            
            '코미디': {
                '로맨스': 0.5,
                '드라마': 0.4,
                '애니메이션': 0.6,
                '가족': 0.5,
                '음악': 0.3
            },
            
            '범죄': {
                '스릴러': 0.8,
                '미스터리': 0.7,
                '드라마': 0.6,
                '액션': 0.5,
                '공포': 0.4
            },
            
            '다큐멘터리': {
                '역사': 0.6,
                '드라마': 0.4,
                '음악': 0.3,
                '전쟁': 0.3
            },
            
            '드라마': {
                '로맨스': 0.7,
                '스릴러': 0.5,
                '코미디': 0.4,
                '범죄': 0.6,
                '미스터리': 0.5,
                '역사': 0.4,
                '다큐멘터리': 0.4,
                '가족': 0.5,
                '음악': 0.4
            },
            
            '가족': {
                '애니메이션': 0.8,
                '코미디': 0.5,
                '모험': 0.6,
                '판타지': 0.5,
                '드라마': 0.5,
                '로맨스': 0.4
            },
            
            '판타지': {
                '모험': 0.7,
                'SF': 0.6,
                '액션': 0.4,
                '애니메이션': 0.5,
                '가족': 0.5,
                '공포': 0.5
            },
            
            '역사': {
                '드라마': 0.4,
                '전쟁': 0.8,
                '다큐멘터리': 0.6,
                '모험': 0.4,
                '서부': 0.5
            },
            
            '공포': {
                '스릴러': 0.7,
                '미스터리': 0.6,
                '판타지': 0.5,
                'SF': 0.4,
                '범죄': 0.4
            },
            
            '음악': {
                '드라마': 0.4,
                '코미디': 0.3,
                '로맨스': 0.5,
                '다큐멘터리': 0.3
            },
            
            '미스터리': {
                '범죄': 0.7,
                '스릴러': 0.8,
                '공포': 0.6,
                '드라마': 0.5
            },
            
            '로맨스': {
                '드라마': 0.7,
                '코미디': 0.5,
                '음악': 0.5,
                '애니메이션': 0.4,
                '가족': 0.4
            },
            
            'SF': {
                '액션': 0.7,
                '모험': 0.6,
                '판타지': 0.6,
                '스릴러': 0.5,
                '공포': 0.4
            },
            
            '스릴러': {
                '범죄': 0.8,
                '미스터리': 0.8,
                '공포': 0.7,
                '액션': 0.6,
                '드라마': 0.5,
                'SF': 0.5
            },
            
            '전쟁': {
                '역사': 0.8,
                '드라마': 0.6,
                '액션': 0.6,
                '다큐멘터리': 0.3
            },
            
            '서부': {
                '액션': 0.5,
                '역사': 0.5,
                '드라마': 0.4
            }
        }
        
        if not target_genres or not candidate_genres:
            return 0.0
        
        # 직접 매치 점수
        direct_match = len(set(target_genres) & set(candidate_genres)) / len(set(target_genres) | set(candidate_genres))
        
        # 간접 매치 점수
        indirect_score = 0
        for target_genre in target_genres:
            for candidate_genre in candidate_genres:
                if target_genre.lower() in genre_similarity_matrix:
                    if candidate_genre.lower() in genre_similarity_matrix[target_genre.lower()]:
                        indirect_score += genre_similarity_matrix[target_genre.lower()][candidate_genre.lower()]
        
        indirect_score = indirect_score / (len(target_genres) * len(candidate_genres)) if target_genres and candidate_genres else 0
        
        return 0.7 * direct_match + 0.3 * indirect_score
    
    def prepare_data(self, movies_df, force_recompute=False):
        """데이터 전처리 및 특성 추출"""
        print("데이터 전처리 중...")
        
        # 필수 컬럼 확인
        required_columns = ['title', 'overview', 'genres']
        for col in required_columns:
            if col not in movies_df.columns:
                raise ValueError(f"필수 컬럼 '{col}'이 데이터에 없습니다.")
        
        # 결측값 처리
        movies_df['title'] = movies_df['title'].fillna('')
        movies_df['overview'] = movies_df['overview'].fillna('')
        movies_df['genres'] = movies_df['genres'].fillna('')
        
        # 키워드 추출
        print("키워드 추출 중...")
        movies_df['keywords'] = movies_df['overview'].apply(self.extract_keywords)
        movies_df['keywords_text'] = movies_df['keywords'].apply(lambda x: ' '.join(x) if x else '')
        
        self.movie_data = movies_df.copy()
        
        processed_title = movies_df['title'].apply(self.enhanced_text_preprocessing_kobert)
        # 1. 제목 TF-IDF 벡터화
        print("제목 TF-IDF 벡터화 중...")
        self.tfidf_title = TfidfVectorizer(
            max_features=5000,
            stop_words=None,
            ngram_range=(1, 2),
            min_df=1
        )
        self.title_matrix = self.tfidf_title.fit_transform(processed_title)

        # 2. 줄거리 TF-IDF 벡터화 (형태소 분리 적용)
        processed_overviews = movies_df['overview'].apply(lambda x: self.enhanced_text_preprocessing_kobert(x))
        
        self.tfidf_overview = TfidfVectorizer(
            max_features=10000, 
            stop_words=None, 
            ngram_range=(1, 2), 
            min_df=1 
        )
        overview_tfidf_matrix = self.tfidf_overview.fit_transform(processed_overviews)
        print("줄거리 TF-IDF 벡터화 완료.")

        # LSA (Truncated SVD)
        self.svd = TruncatedSVD(n_components=100, random_state=42)
        self.overview_matrix = self.svd.fit_transform(overview_tfidf_matrix)
        
        # 3. 키워드 TF-IDF 벡터화
        print("키워드 TF-IDF 벡터화 중...")
        self.tfidf_keywords = TfidfVectorizer(
            max_features=1000,
            stop_words=None,
            ngram_range=(1, 1),
            min_df=3,  # 최소 2개 영화에서 나타나는 키워드만 사용
            max_df=0.4,  # 40% 이상 영화에 나타나는 키워드는 제외
        )
        self.keyword_matrix = self.tfidf_keywords.fit_transform(movies_df['keywords_text'])
        
        # 4. 장르 처리
        print("장르 처리 중...")
        genre_lists = []
        for genres in movies_df['genres']:
            if isinstance(genres, str) and genres:
                genre_list = [g.strip() for g in genres.split(',')]
                genre_lists.append(genre_list)
            elif isinstance(genres, list):
                genre_lists.append(genres)
            else:
                genre_lists.append([])
        
        self.mlb_genre = MultiLabelBinarizer()
        self.genre_matrix = self.mlb_genre.fit_transform(genre_lists)

        # 5. KoBERT 임베딩
        if not force_recompute and os.path.exists("plot_embeddings.npy"):
            print("✅ 저장된 줄거리 임베딩을 불러옵니다.")
            self.plot_embeddings = np.load("plot_embeddings.npy")
        else:
            print("🛠 줄거리 임베딩을 새로 생성합니다.")
            if self.kobert_model is None:
                self.load_kobert_model()
        
            plot_embeddings = []
            for i, overview in enumerate(processed_overviews):
                if i % 50 == 0:
                    print(f"임베딩 진행률: {i}/{len(movies_df)}")
                embedding = self.get_kobert_embedding(overview)
                plot_embeddings.append(embedding)

            self.plot_embeddings = np.array(plot_embeddings)
            np.save("plot_embeddings.npy", self.plot_embeddings)
            print("줄거리 임베딩 저장 완료: plot_embeddings.npy")
            print("데이터 전처리 완료")
    
    def calculate_similarity_comprehensive(self, movie_idx, weights):
        """포괄적인 유사도 계산"""
        
        # 1. KoBERT 임베딩 기반 유사도 (의미적 유사도)
        target_plot_embedding = self.plot_embeddings[movie_idx].reshape(1, -1)
        kobert_similarities = cosine_similarity(target_plot_embedding, self.plot_embeddings)[0]
        kobert_similarities[kobert_similarities < 0] = 0
        
        # 2. 줄거리 TF-IDF 기반 유사도 (어휘적 유사도)
        target_overview_vector = self.overview_matrix[movie_idx]
        overview_similarities = cosine_similarity(target_overview_vector.reshape(1, -1), self.overview_matrix)[0]
        overview_similarities[overview_similarities < 0] = 0
        
        # 3. 제목 유사도
        target_title_vector = self.title_matrix[movie_idx]
        title_similarities = cosine_similarity(target_title_vector, self.title_matrix)[0]
        
        # 4. 키워드 유사도
        target_keyword_vector = self.keyword_matrix[movie_idx]
        keyword_similarities = cosine_similarity(target_keyword_vector, self.keyword_matrix)[0]
        
        # 5. 고급 장르 유사도
        target_genres = self.movie_data.iloc[movie_idx]['genres'].split(',') if self.movie_data.iloc[movie_idx]['genres'] else []
        target_genres = [g.strip() for g in target_genres]
        
        genre_similarities = []
        for i in range(len(self.movie_data)):
            candidate_genres = self.movie_data.iloc[i]['genres'].split(',') if self.movie_data.iloc[i]['genres'] else []
            candidate_genres = [g.strip() for g in candidate_genres]
            
            similarity = self.calculate_genre_similarity_advanced(target_genres, candidate_genres)
            genre_similarities.append(similarity)
        
        genre_similarities = np.array(genre_similarities)
        
        # 6. 가중 평균으로 최종 유사도 계산
        final_similarities = (
            weights['kobert'] * kobert_similarities +
            weights['overview_tfidf'] * overview_similarities +
            weights['title'] * title_similarities +
            weights['keywords'] * keyword_similarities +
            weights['genre'] * genre_similarities
        )
        
        return (final_similarities, kobert_similarities, overview_similarities, 
                title_similarities, keyword_similarities, genre_similarities)
    
    def adaptive_weights(self, movie_idx):
        """영화 특성에 따른 적응적 가중치 계산"""
        movie_info = self.movie_data.iloc[movie_idx]
        
        # 1. 기본 가중치 설정 (총합 100%)
        weights = {
            'kobert': 0.15,
            'overview_tfidf': 0.25,
            'title': 0.05,  # 제목 유사도 문제 해결 전까지는 낮게 유지
            'keywords': 0.25, # 키워드 중요성을 높임
            'genre': 0.30   # 장르 중요성을 높임
        }

        # 모든 가중치 조정은 상대적으로 이루어지므로, 총합이 1이 되도록 조정하는 것이 중요합니다.
        # 여기서는 편의상 절대값으로 더하고 빼지만, 실제로는 비율로 조정하거나, 마지막에 정규화해야 합니다.


        # 2. 줄거리 길이에 따른 조정
        overview_length = len(movie_info['overview']) if movie_info['overview'] else 0

        if overview_length > 300:  # 긴 줄거리: 줄거리 관련 가중치 강화
            weights['kobert'] += 0.03
            weights['overview_tfidf'] += 0.05
            weights['genre'] -= 0.04 # 상대적으로 장르 중요도 감소
            weights['keywords'] -= 0.04
        elif overview_length < 50:  # 짧은 줄거리: 제목/장르/키워드 강화, 줄거리 관련 약화
            weights['title'] += 0.05 # 제목 유사도 개선 시 효과적
            weights['genre'] += 0.05
            weights['keywords'] += 0.05
            weights['kobert'] -= 0.07 # KoBERT 낮춤
            weights['overview_tfidf'] -= 0.08 # TF-IDF 낮춤

        # 3. 장르 수에 따른 조정 (if-elif-else 구조 사용)
        genre_count = len(movie_info['genres'].split(',')) if movie_info['genres'] else 0

        if genre_count == 1:  # 단일 장르: 해당 장르의 특성을 강하게 반영
            weights['genre'] += 0.10
            weights['overview_tfidf'] -= 0.05
            weights['keywords'] -= 0.05
            weights['kobert'] -= 0.02
        elif genre_count > 2:  # 많은 장르: 특정 장르에 덜 치우치도록
            weights['genre'] -= 0.03
            weights['kobert'] += 0.03 # KoBERT 영향력을 다시 높여 전체 줄거리 문맥 반영
        # else: (2개의 장르인 경우 기본 가중치 유지)

        # 4. 특정 장르에 대한 조정 (가장 구체적인 조건부터 배치)
        genres = movie_info['genres'].lower() if movie_info['genres'] else ''

        # 스릴러와 드라마 조합 (기생충, 하녀 등)
        if '드라마' in genres and '스릴러' in genres and '코미디' in genres: # 기생충 같은 경우
            weights['genre'] += 0.15 # 장르 매우 중요
            weights['keywords'] += 0.05
            weights['kobert'] -= 0.05 # KoBERT 비중 감소
        elif '드라마' in genres and '스릴러' in genres: # 일반적인 드라마/스릴러
            weights['genre'] += 0.10
            weights['keywords'] += 0.05
            weights['overview_tfidf'] += 0.02 # 줄거리 핵심 단어 중요도도 높임
        elif 'sf' in genres or '공포' in genres: # '괴물' 같은 경우
            weights['genre'] += 0.15 # 장르 매우 중요
            weights['keywords'] += 0.10 # SF/공포는 키워드가 중요 (괴물, 좀비 등)
            weights['overview_tfidf'] += 0.05 # 줄거리 내 핵심 단어 중요
            weights['kobert'] -= 0.10 # KoBERT 비중 대폭 감소
        elif '액션' in genres:
            weights['keywords'] += 0.05
            weights['overview_tfidf'] += 0.05 # 액션은 줄거리 내 동작 묘사가 중요
            weights['title'] -= 0.05 # 제목보다는 내용이 중요
        elif '로맨스' in genres:
            weights['kobert'] += 0.03 # 감성적인 줄거리 유사성 중요
            weights['overview_tfidf'] += 0.03
            weights['keywords'] -= 0.03
            weights['genre'] -= 0.03 # 로맨스는 장르가 넓어질 수 있으므로
        elif '코미디' in genres:
            weights['kobert'] += 0.02 # 코미디도 줄거리의 뉘앙스가 중요
            weights['genre'] += 0.05 # 코미디 장르의 특색
            weights['keywords'] -= 0.02

        # 음수 가중치 방지 및 최소값 설정 (선택 사항)
        for key in weights:
            if weights[key] < 0.01: # 최소 가중치 설정 (예: 1%)
                weights[key] = 0.01

        # 가중치 정규화
        total_weight = sum(weights.values())
        weights = {k: v/total_weight for k, v in weights.items()}
        
        return weights
    
    def recommend_movies(self, movie_title=None, movie_idx=None, top_n=10, weights=None, use_adaptive_weights=True):
        """영화 추천"""
        
        if movie_idx is None:
            if movie_title is None:
                raise ValueError("movie_title 또는 movie_idx 중 하나는 제공되어야 합니다.")
            
            # 제목으로 인덱스 찾기
            matches = self.movie_data[self.movie_data['title'].str.contains(movie_title, case=False, na=False)]
            if matches.empty:
                print(f"'{movie_title}'과 일치하는 영화를 찾을 수 없습니다.")
                return None
            
            movie_idx = matches.index[0]
            actual_title = matches.iloc[0]['title']
        else:
            actual_title = self.movie_data.iloc[movie_idx]['title']
        
        # 가중치 결정
        if use_adaptive_weights:
            weights = self.adaptive_weights(movie_idx)
            print(f"적응적 가중치 사용: {weights}")
        elif weights is None:
            weights = {
                'kobert': 0.15,
                'overview_tfidf': 0.35,
                'title': 0.10,
                'keywords': 0.25,
                'genre': 0.15
            }
        
        print(f"\n기준 영화: {actual_title}")
        print(f"장르: {self.movie_data.iloc[movie_idx]['genres']}")
        print(f"줄거리: {self.movie_data.iloc[movie_idx]['overview'][:400]}...")
        print(f"   키워드: {self.movie_data.iloc[movie_idx]['keywords']}")
        
        # 유사도 계산
        similarities = self.calculate_similarity_comprehensive(movie_idx, weights)
        final_similarities = similarities[0]
        
        # 자기 자신 제외하고 상위 N개 추천
        similar_indices = np.argsort(final_similarities)[::-1][1:top_n+1]
        
        # 추천 결과 생성
        recommendations = []
        for idx in similar_indices:
            movie_info = {
                'title': self.movie_data.iloc[idx]['title'],
                'genres': self.movie_data.iloc[idx]['genres'],
                'overview': self.movie_data.iloc[idx]['overview'],
                'keywords': self.movie_data.iloc[idx]['keywords'],
                'total_similarity': final_similarities[idx],
                'kobert_similarity': similarities[1][idx],
                'overview_tfidf_similarity': similarities[2][idx],
                'title_similarity': similarities[3][idx],
                'keyword_similarity': similarities[4][idx],
                'genre_similarity': similarities[5][idx]
            }
            recommendations.append(movie_info)
        
        return recommendations
    
    def display_recommendations(self, recommendations):
        """추천 결과 출력"""
        if not recommendations:
            print("추천할 영화가 없습니다.")
            return
        
        print(f"\n=== 추천 영화 TOP {len(recommendations)} ===")
        
        for i, movie in enumerate(recommendations, 1):
            print(f"\n{i}. {movie['title']}")
            print(f"   장르: {movie['genres']}")
            print(f"   총 유사도: {movie['total_similarity']:.3f}")
            print(f"   세부 유사도:")
            print(f"     - KoBERT: {movie['kobert_similarity']:.3f}")
            print(f"     - 줄거리 TF-IDF: {movie['overview_tfidf_similarity']:.3f}")
            print(f"     - 제목: {movie['title_similarity']:.3f}")
            print(f"     - 키워드: {movie['keyword_similarity']:.3f}")
            print(f"     - 장르: {movie['genre_similarity']:.3f}")
            print(f"   키워드: {movie['keywords']}")
            print(f"   줄거리: {movie['overview'][:100]}...")

In [ ]:
# 사용 예시
def create_extended_sample_data():
    # movie_data.csv 파일을 불러와서 DataFrame 반환
    df = pd.read_csv('../data_processing/movie_data.csv', encoding='utf-8-sig')
    return df

# 실행 예시
if __name__ == "__main__":
    # 확장된 샘플 데이터 생성
    movies_df = create_extended_sample_data()
    
    # 개선된 추천 시스템 초기화
    recommender = ImprovedMovieRecommendationSystem()
    
    # 데이터 전처리
    recommender.prepare_data(movies_df)
    
    # 적응적 가중치로 추천
    print("=== 괴물 기반 추천 (적응적 가중치) ===")
    recommendations1 = recommender.recommend_movies(
        movie_title='괴물',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations1)
    
    # 다른 영화로 테스트
    print("\n=== 기생충 기반 추천 (적응적 가중치) ===")
    recommendations2 = recommender.recommend_movies(
        movie_title='기생충',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations2)

    # 다른 영화로 테스트
    print("\n=== 어벤져스 기반 추천 (적응적 가중치) ===")
    recommendations3 = recommender.recommend_movies(
        movie_title='어벤져스',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations3)

    print("\n=== 유열의 음악앨범 기반 추천 (적응적 가중치) ===")
    recommendations4 = recommender.recommend_movies(
        movie_title='유열의 음악앨범',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations4)

    print("\n=== 완벽한 타인 기반 추천 (적응적 가중치) ===")
    recommendations5 = recommender.recommend_movies(
        movie_title='완벽한 타인',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations5)

불용어 파일 로드 완료: ./stopwords/git_stopwords.txt (595개)
데이터 전처리 중...
✅ 저장된 특성 데이터를 불러옵니다...
✅ 모든 특성 데이터 로드 완료.
=== 괴물 기반 추천 (적응적 가중치) ===
적응적 가중치 사용: {'kobert': 0.09166666666666666, 'overview_tfidf': 0.3333333333333333, 'title': 0.08333333333333334, 'keywords': 0.21666666666666667, 'genre': 0.27499999999999997}

기준 영화: 괴물
장르: SF, 공포, 드라마
줄거리: 아버지가 운영하는 한강매점, 늘어지게 낮잠 자던 강두는 우연히 특이한 광경을 목격하게 된다. 생전 보도 못한 무언가가 한강다리에 매달려 움직이는 것이다. 정체를 알 수 없는 괴물은 둔치 위로 올라와 사람들을 거침없이 깔아뭉개고, 무차별로 물어뜯기 시작한다. 순식간에 아수라장으로 돌변하는 한강변. 강두도 뒤늦게 딸 현서를 데리고 정신없이 도망가지만, 꼭 잡았던 현서의 손을 놓치고 만다. 하루아침에 집과 생계, 그리고 현서까지 모든 것을 잃게 된 강두 가족. 돈도 없고 빽도 없는 그들은 위험구역으로 선포된 한강 어딘가에 있을 현서를 찾아 나선다....
   키워드: ['괴물', '강두', '매점', '낮잠', '광경', '생전']

=== 추천 영화 TOP 8 ===

1. 초능력자
   장르: SF, 스릴러, 액션
   총 유사도: 0.341
   세부 유사도:
     - KoBERT: 0.877
     - 줄거리 TF-IDF: 0.342
     - 제목: 0.000
     - 키워드: 0.446
     - 장르: 0.180
   키워드: ['싸움', '괴물', '초인', '규남', '사람', '작고', '외진']
   줄거리: 규남이 일하는 작고 외진 전당포, ‘유토피아’. 돈을 훔치러 들어온 초인이 사람들을 조종하기 시작하지만 초인의 통제를 벗어나 누군

In [26]:
# 키워드 추천 개선
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import re
import os, pickle, json
from collections import Counter
from konlpy.tag import Okt

class ImprovedMovieRecommendationSystem:
    def __init__(self, stopword_files = None, cache_dir="./cached_features"):
        self.kobert_tokenizer = None
        self.kobert_model = None
        self.tfidf_title = None
        self.tfidf_overview = None
        self.tfidf_keywords = None
        self.mlb_genre = None
        self.movie_data = None
        self.plot_embeddings = None
        self.title_matrix = None
        self.genre_matrix = None
        self.keyword_matrix = None
        self.overview_matrix = None
        
        self.cache_dir = cache_dir

        # 캐시 디렉토리 생성
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
            print(f"캐시 디렉토리 생성: {self.cache_dir}")

        # 불용어 파일 리스트가 제공되면 로드, 아니면 기본 빈 set
        if stopword_files is None:
            # 기본 불용어 파일 (예시: 프로젝트 내에 git_stopwords.txt 하나만 있는 경우)
            # 또는 비어있는 set으로 시작하여 불용어 처리를 하지 않을 수도 있습니다.
            stopword_files = ["git_stopwords.txt"] # 기본 파일 경로 지정
        
        self.custom_stopwords = self.load_stopwords_from_files(stopword_files)
        
    def load_kobert_model(self):
        """KoBERT 모델 로드"""
        print("KoBERT 모델 로딩 중...")
        model_name = "monologg/kobert"
        self.kobert_model = AutoModel.from_pretrained(model_name)
        self.kobert_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.kobert_model.eval()
        print("KoBERT 모델 로드 완료")

    def enhanced_text_preprocessing(self, text):
        """텍스트 전처리 강화"""
        if not text or pd.isna(text):
            return ""
        
        text = str(text)
        
        # 특수 문자 및 숫자 처리
        text = re.sub(r'[^\w\s가-힣a-zA-Z0-9]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()

    def extract_nouns_from_text(self, text, okt):
        """텍스트에서 명사 추출"""
        if not text:
            return []
        
        # 명사 추출
        nouns = okt.nouns(text)
        
        # 단어 길이가 1 이하인 것 제거
        filtered_nouns = [noun for noun in nouns if len(noun) > 1]
        
        return filtered_nouns
    
    def load_stopwords_from_files(self, stopword_files):
        """여러 불용어 파일을 로드하여 하나의 set으로 통합"""
        combined_stopwords = set()
        
        base_dir = "./stopwords/"

        for file_name in stopword_files:
            file_path = os.path.join(base_dir, file_name)

            if os.path.exists(file_path):
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        words = [line.strip() for line in f if line.strip()]
                        combined_stopwords.update(words)
                    print(f"불용어 파일 로드 완료: {file_path} ({len(words)}개)")
                except Exception as e:
                    print(f"불용어 파일 로드 실패: {file_path} - {e}")
    
        if not combined_stopwords: # 로드된 불용어가 없을 경우 (파일이 없거나 비어있을 때)
            print("❗ 모든 불용어 파일을 로드할 수 없거나 파일이 비어 있어 기본 불용어를 사용합니다.")
            # 예비 불용어 리스트를 set으로 변환
            return set(['하다', '있다', '없다', '되다', '이다', '것', '수', '점', '말', '안', '때', '등', '통해'])
        
        return combined_stopwords


    def enhanced_text_preprocessing_token(self, text):
        if pd.isna(text) or not text:
            return ""

        text = self.enhanced_text_preprocessing(text)
        stopwords_pos = ['Josa', 'Eomi', 'Suffix', 'Punctuation'] 
        
        tokens = self.okt.pos(text, norm=True, stem=True)
        
        filtered_tokens = []
        for word, pos in tokens:
            if pos not in stopwords_pos:
                # 함수 내부에서 로드한 custom_stopwords 사용
                if word not in self.custom_stopwords: 
                    filtered_tokens.append(word)
        
        return ' '.join(filtered_tokens)
    
    def extract_keywords(self, text):
        """텍스트에서 키워드 추출 (형태소 분석 + 불용어 제거 + 가중치 적용)"""
        if not text or pd.isna(text):
            return []
        
        # OKT 형태소 분석기 초기화 (클래스 변수로 관리하면 더 효율적)
        if not hasattr(self, 'okt'):
            self.okt = Okt()
        
        # 불용어 로드 (처음 한 번만 실행)
        if not hasattr(self, 'stopwords'):
            stopword_files = [
                'common_name_stopwords.txt',
                'korean_adverb_stopwords.txt', 
                'general_stopwords.txt',
                'git_stopwords.txt'
            ]
            self.stopwords = self.load_stopwords_from_files(stopword_files)
            print(f"전체 불용어 개수: {len(self.stopwords)}")
        
        # 한국어 영화 관련 키워드 패턴 (가중치 포함)
        movie_keywords = {
            # 장르 관련 (가중치 2)
            'action': {
                'weight': 2,
                'keywords': [
                    '액션', '전투', '싸움', '전쟁', '격투', '추격', '폭발', '총격',
                    '무술', '카체이싱', '건파이트', '배틀', '액션스릴러', '느와르',
                    '스파이', '첩보', '잠입', '미션', '작전', '복수', '정의', '위장'
                ]
            },
            'romance': {
                'weight': 2,
                'keywords': [
                    '사랑', '로맨스', '연인', '결혼', '연애', '첫사랑', '이별', '만남',
                    '멜로', '러브스토리', '운명', '재회', '프로포즈', '웨딩', '데이트',
                    '썸', '고백', '짝사랑', '원거리', '국제연애', '나이차', '사내연애'
                ]
            },
            'comedy': {
                'weight': 2,
                'keywords': [
                    '코미디', '웃음', '유머', '재미', '개그', '유쾌', '농담',
                    '개그맨', '상황극', '슬랩스틱', '로맨틱코미디', '패밀리코미디',
                    '블랙코미디', '풍자', '해학', '익살', '코믹', '유머러스'
                ]
            },
            'horror': {
                'weight': 2,
                'keywords': [
                    '공포', '호러', '무서운', '귀신', '좀비', '괴물', '악령',
                    '사이코', '살인마', '연쇄살인', '초자연', '오컬트', '엑소시즘',
                    '저주', '원혼', '귀신', '유령', '무덤', '폐가', '심령', '위험구역'
                ]
            },
            'thriller': {
                'weight': 2,
                'keywords': [
                    '스릴러', '긴장', '추적', '수사', '범죄', '살인', '미스터리',
                    '서스펜스', '추리', '탐정', '형사', '검찰', '법정', '재판',
                    '납치', '협박', '음모', '배신', '사기', '해킹', '첩보'
                ]
            },
            'drama': {
                'weight': 2,
                'keywords': [
                    '인간', '감동', '눈물', '인생', '성장'
                ]
            },
            
            # 테마/소재 관련 (가중치 1.5)
            'theme': {
                'weight': 1.5,
                'keywords': [
                    '복수', '정의', '우정', '배신', '희생',
                    '성장', '자아실현', '꿈', '도전', '극복', '화해',
                    '갈등', '대립', '협력', '경쟁', '성공', '실패'
                ]
            },
            'emotion': {
                'weight': 1.5,
                'keywords': [
                    '감동', '눈물', '슬픔', '기쁨', '행복', '분노',
                    '절망', '희망', '사랑', '이별', '그리움', '향수',
                    '외로움', '고독', '우울', '스트레스', '끔찍한'
                ]
            },
            
            # 배경/직업 관련 (가중치 1)
            'profession': {
                'weight': 1,
                'keywords': [
                    '의사', '변호사', '교사', '경찰', '소방관', '군인',
                    '기자', '작가', '화가', '음악가', '요리사', '사업가',
                    '정치인', '연예인', '운동선수', '과학자', '엔지니어'
                ]
            },
            'background': {
                'weight': 1,
                'keywords': [
                    '현대', '도시', '서울', '학교', '대학', '회사', '직장',
                    '병원', '법원', '경찰서', '아파트', '카페'
                ]
            }
        }
        
        # 1. 텍스트 전처리
        preprocessed_text = self.enhanced_text_preprocessing(text)
        
        # 2. 형태소 분석으로 명사 추출
        nouns = self.extract_nouns_from_text(preprocessed_text, self.okt)
        
        # 3. 불용어 제거
        filtered_nouns = [noun for noun in nouns 
                        if noun not in self.stopwords]
        
        # 4. 명사 빈도 계산
        noun_counts = Counter(filtered_nouns)
        
        # 5. 영화 키워드 매칭 및 가중치 적용
        keyword_scores = {}
        
        for category, category_info in movie_keywords.items():
            weight = category_info['weight']
            keywords = category_info['keywords']
            
            for keyword in keywords:
                # 키워드가 추출된 명사에 있는지 확인
                if keyword in filtered_nouns:
                    score = noun_counts[keyword] * weight
                    if keyword in keyword_scores:
                        keyword_scores[keyword] += score
                    else:
                        keyword_scores[keyword] = score
        
        # 6. 점수 기준으로 정렬하여 상위 키워드 반환
        sorted_keywords = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)
        
        # 상위 20개 키워드만 반환 (키워드만, 점수 제외)
        top_keywords = [keyword for keyword, score in sorted_keywords[:20]]
        
        # 매칭되지 않은 중요 명사도 일부 포함 (빈도수 기준)
        remaining_nouns = [noun for noun in noun_counts.most_common(10) 
                        if noun[0] not in top_keywords]
        
        # 최종 키워드 결합
        final_keywords = top_keywords + [noun[0] for noun in remaining_nouns[:5]]
        
        return final_keywords[:25]  # 최대 25개 키워드 반환


    def get_kobert_embedding(self, text):
        """KoBERT를 사용하여 텍스트 임베딩 생성"""
        if not text or pd.isna(text):
            return np.zeros(768)  # KoBERT 임베딩 차원
        
        # 텍스트 길이 제한 (KoBERT 최대 입력 길이 고려)
        text = str(text)[:512]
        
        inputs = self.kobert_tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )
        
        with torch.no_grad():
            outputs = self.kobert_model(**inputs)
            # [CLS] 토큰의 임베딩 사용
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().detach().cpu().numpy()
        
        return embedding
    
    def calculate_genre_similarity_advanced(self, target_genres, candidate_genres):
        """고급 장르 유사도 계산"""
        # 장르 간 유사도 매트릭스 (실제로는 더 정교하게 구성 가능)
        with open("./genre_similarity_matrix_content.json", "r", encoding="utf-8") as f:
            genre_similarity_matrix = json.load(f)
    
        if not target_genres or not candidate_genres:
            return 0.0
        
        # 직접 매치 점수
        direct_match = len(set(target_genres) & set(candidate_genres)) / len(set(target_genres) | set(candidate_genres))
        
        # 간접 매치 점수
        indirect_score = 0
        for target_genre in target_genres:
            for candidate_genre in candidate_genres:
                if target_genre.lower() in genre_similarity_matrix:
                    if candidate_genre.lower() in genre_similarity_matrix[target_genre.lower()]:
                        indirect_score += genre_similarity_matrix[target_genre.lower()][candidate_genre.lower()]
        
        indirect_score = indirect_score / (len(target_genres) * len(candidate_genres)) if target_genres and candidate_genres else 0
        
        return 0.7 * direct_match + 0.3 * indirect_score
    
    def prepare_data(self, movies_df, force_recompute=False):
        """데이터 전처리 및 특성 추출"""
        print("데이터 전처리 중...")

         # 캐시 파일 경로 정의
        feature_objects_path = f"{self.cache_dir}/feature_objects.pkl"
        feature_matrices_path = f"{self.cache_dir}/feature_matrices.npz"
        movie_data_path = f"{self.cache_dir}/movie_data.pkl"
        plot_embeddings_path = f"{self.cache_dir}/plot_embeddings.npy" # KoBERT 임베딩도 여기에 포함

        # 캐시 존재 여부 확인 및 force_recompute에 따른 로드/재계산 결정
        if not force_recompute and \
           os.path.exists(feature_objects_path) and \
           os.path.exists(feature_matrices_path) and \
           os.path.exists(movie_data_path) and \
           os.path.exists(plot_embeddings_path): # KoBERT 임베딩 파일도 확인
            
            print("✅ 저장된 특성 데이터를 불러옵니다...")
            try:
                # 1. Feature objects (Vectorizer, SVD, MLB) 로드
                with open(feature_objects_path, 'rb') as f:
                    feature_objects = pickle.load(f)
                    self.tfidf_title = feature_objects['tfidf_title']
                    self.tfidf_overview = feature_objects['tfidf_overview']
                    self.tfidf_keywords = feature_objects['tfidf_keywords']
                    self.svd = feature_objects['svd']
                    self.mlb_genre = feature_objects['mlb_genre']

                # 2. Feature matrices (NumPy arrays) 로드
                loaded_matrices = np.load(feature_matrices_path, allow_pickle=True)
                self.title_matrix = loaded_matrices['title_matrix']
                self.overview_matrix = loaded_matrices['overview_matrix']
                self.keyword_matrix = loaded_matrices['keyword_matrix']
                self.genre_matrix = loaded_matrices['genre_matrix']

                # 3. KoBERT 임베딩 로드 (별도 파일로 관리)
                self.plot_embeddings = np.load(plot_embeddings_path)

                # 4. 영화 데이터 로드
                self.movie_data = pd.read_pickle(movie_data_path)

                print("✅ 모든 특성 데이터 로드 완료.")
                return

            except Exception as e:
                print(f"❌ 저장된 특성 데이터 로드 중 오류 발생: {e}. 데이터를 새로 계산합니다.")
                # 오류 발생 시 다시 계산하도록 플래그 설정
                force_recompute = True # 에러 발생 시 재계산을 유도

        # 캐시가 없거나, 강제로 재계산해야 하는 경우
        print("🛠 데이터를 새로 전처리하고 특성을 추출합니다...")
        
        # 필수 컬럼 확인
        required_columns = ['title', 'overview', 'genres']
        for col in required_columns:
            if col not in movies_df.columns:
                raise ValueError(f"필수 컬럼 '{col}'이 데이터에 없습니다.")
        
        # 결측값 처리
        movies_df['title'] = movies_df['title'].fillna('')
        movies_df['overview'] = movies_df['overview'].fillna('')
        movies_df['genres'] = movies_df['genres'].fillna('')
        
        # 키워드 추출
        print("키워드 추출 중...")
        movies_df['keywords'] = movies_df['overview'].apply(self.extract_keywords)
        movies_df['keywords_text'] = movies_df['keywords'].apply(lambda x: ' '.join(x) if x else '')
        
        movies_df['title_text'] = movies_df['title'].apply(self.enhanced_text_preprocessing_token)
        movies_df['overview_text'] = movies_df['overview'].apply(self.enhanced_text_preprocessing_token)

        self.movie_data = movies_df.copy()
        
        # 1. 제목 TF-IDF 벡터화
        print("제목 TF-IDF 벡터화 중...")
        self.tfidf_title = TfidfVectorizer(
            max_features=5000,
            stop_words=None,
            ngram_range=(1, 2),
            min_df=1
        )
        self.title_matrix = self.tfidf_title.fit_transform(movies_df['title_text'])

        # 2. 줄거리 TF-IDF 벡터화 (형태소 분리 적용)    
        self.tfidf_overview = TfidfVectorizer(
            max_features=10000, 
            stop_words=None, 
            ngram_range=(1, 2), 
            min_df=1 
        )
        overview_tfidf_matrix = self.tfidf_overview.fit_transform(movies_df['overview_text'])
        print("줄거리 TF-IDF 벡터화 완료.")

        # LSA (Truncated SVD)
        self.svd = TruncatedSVD(n_components=100, random_state=42)
        self.overview_matrix = self.svd.fit_transform(overview_tfidf_matrix)
        
        # 3. 키워드 TF-IDF 벡터화
        print("키워드 TF-IDF 벡터화 중...")
        self.tfidf_keywords = TfidfVectorizer(
            max_features=1000,
            stop_words=None,
            ngram_range=(1, 1),
            min_df=3,  # 최소 2개 영화에서 나타나는 키워드만 사용
            max_df=0.4,  # 40% 이상 영화에 나타나는 키워드는 제외
        )
        self.keyword_matrix = self.tfidf_keywords.fit_transform(movies_df['keywords_text'])
        
        # 4. 장르 처리
        print("장르 처리 중...")
        genre_lists = []
        for genres in movies_df['genres']:
            if isinstance(genres, str) and genres:
                genre_list = [g.strip() for g in genres.split(',')]
                genre_lists.append(genre_list)
            elif isinstance(genres, list):
                genre_lists.append(genres)
            else:
                genre_lists.append([])
        
        self.mlb_genre = MultiLabelBinarizer()
        self.genre_matrix = self.mlb_genre.fit_transform(genre_lists)

        # 5. KoBERT 임베딩
        print("🛠 줄거리 임베딩을 새로 생성합니다.")
        if self.kobert_model is None:
            self.load_kobert_model()
        
        plot_embeddings = []
        for i, overview in enumerate(movies_df['overview_text']):
            if i % 50 == 0:
                print(f"임베딩 진행률: {i}/{len(movies_df)}")
            embedding = self.get_kobert_embedding(overview)
            plot_embeddings.append(embedding)

        self.plot_embeddings = np.array(plot_embeddings)
        print("줄거리 임베딩 생성 완료.")

        self._save_all_features()
        print("데이터 전처리 완료")

    def _save_all_features(self):
        """모든 특성을 한 번에 저장"""
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
        
        print("💾 모든 특성 데이터 저장 중...")
        
        # 1. TF-IDF 벡터화 객체들 저장
        feature_objects = {
            'tfidf_title': self.tfidf_title,
            'tfidf_overview': self.tfidf_overview,
            'tfidf_keywords': self.tfidf_keywords,
            'svd': self.svd,
            'mlb_genre': self.mlb_genre
        }
        
        # Ensure file exists before attempting to open, or handle errors
        try:
            with open(f"{self.cache_dir}/feature_objects.pkl", 'wb') as f:
                pickle.dump(feature_objects, f)
        except Exception as e:
            print(f"Error saving feature objects: {e}")
            # Optionally, re-raise or handle more gracefully

        # 2. 특성 행렬들 저장
        # Scipy sparse matrix (CSR) toarray() to save as dense numpy array
        # Check if matrices are sparse before converting
        title_matrix_array = self.title_matrix.toarray() if hasattr(self.title_matrix, 'toarray') else self.title_matrix
        keyword_matrix_array = self.keyword_matrix.toarray() if hasattr(self.keyword_matrix, 'toarray') else self.keyword_matrix
        overview_matrix_array = self.overview_matrix if isinstance(self.overview_matrix, np.ndarray) else self.overview_matrix.toarray()


        np.savez_compressed(f"{self.cache_dir}/feature_matrices.npz",
                            title_matrix=title_matrix_array,
                            overview_matrix=overview_matrix_array,
                            keyword_matrix=keyword_matrix_array,
                            genre_matrix=self.genre_matrix)
        
        # KoBERT 임베딩은 별도 파일로 저장
        np.save(f"{self.cache_dir}/plot_embeddings.npy", self.plot_embeddings)

        # 3. 영화 데이터 저장 (전처리된 컬럼 포함)
        self.movie_data.to_pickle(f"{self.cache_dir}/movie_data.pkl")
        
        print(f"✅ 모든 데이터가 {self.cache_dir}에 저장되었습니다.")
    
    def calculate_similarity_comprehensive(self, movie_idx, weights):
        """포괄적인 유사도 계산"""
        
        # 1. KoBERT 임베딩 기반 유사도 (의미적 유사도)
        target_plot_embedding = self.plot_embeddings[movie_idx].reshape(1, -1)
        kobert_similarities = cosine_similarity(target_plot_embedding, self.plot_embeddings)[0]
        kobert_similarities[kobert_similarities < 0] = 0
        
        # 2. 줄거리 TF-IDF 기반 유사도 (어휘적 유사도)
        target_overview_vector = self.overview_matrix[movie_idx]
        overview_similarities = cosine_similarity(target_overview_vector.reshape(1, -1), self.overview_matrix)[0]
        overview_similarities[overview_similarities < 0] = 0
        
        # 3. 제목 유사도
        target_title_vector = self.title_matrix[movie_idx]
        title_similarities = cosine_similarity(target_title_vector.reshape(1, -1), self.title_matrix)[0]
        
        # 4. 키워드 유사도
        target_keyword_vector = self.keyword_matrix[movie_idx]
        keyword_similarities = cosine_similarity(target_keyword_vector.reshape(1, -1), self.keyword_matrix)[0]
        
        # 5. 장르 유사도
        target_genres = self.movie_data.iloc[movie_idx]['genres'].split(',') if self.movie_data.iloc[movie_idx]['genres'] else []
        target_genres = [g.strip() for g in target_genres]
        
        genre_similarities = []
        for i in range(len(self.movie_data)):
            candidate_genres = self.movie_data.iloc[i]['genres'].split(',') if self.movie_data.iloc[i]['genres'] else []
            candidate_genres = [g.strip() for g in candidate_genres]
            
            similarity = self.calculate_genre_similarity_advanced(target_genres, candidate_genres)
            genre_similarities.append(similarity)
        
        genre_similarities = np.array(genre_similarities)
        
        # 6. 가중 평균으로 최종 유사도 계산
        final_similarities = (
            weights['kobert'] * kobert_similarities +
            weights['overview_tfidf'] * overview_similarities +
            weights['title'] * title_similarities +
            weights['keywords'] * keyword_similarities +
            weights['genre'] * genre_similarities
        )
        
        return (final_similarities, kobert_similarities, overview_similarities, 
                title_similarities, keyword_similarities, genre_similarities)
    
    def adaptive_weights(self, movie_idx):
        """영화 특성에 따른 적응적 가중치 계산"""
        movie_info = self.movie_data.iloc[movie_idx]
        
        # 1. 기본 가중치 설정 (총합 100%)
        weights = {
            'kobert': 0.20,
            'overview_tfidf': 0.35,
            'title': 0.10,  # 제목 유사도 문제 해결 전까지는 낮게 유지
            'keywords': 0.20, # 키워드 중요성을 높임
            'genre': 0.25   # 장르 중요성을 높임
        }

        # 모든 가중치 조정은 상대적으로 이루어지므로, 총합이 1이 되도록 조정하는 것이 중요합니다.
        # 여기서는 편의상 절대값으로 더하고 빼지만, 실제로는 비율로 조정하거나, 마지막에 정규화해야 합니다.


        # 2. 줄거리 길이에 따른 조정
        overview_length = len(movie_info['overview']) if movie_info['overview'] else 0

        if overview_length > 300:  # 긴 줄거리: 줄거리 관련 가중치 강화
            weights['kobert'] += 0.03
            weights['overview_tfidf'] += 0.05
            weights['genre'] -= 0.04 # 상대적으로 장르 중요도 감소
            weights['keywords'] -= 0.04
        elif overview_length < 50:  # 짧은 줄거리: 제목/장르/키워드 강화, 줄거리 관련 약화
            weights['title'] += 0.05 # 제목 유사도 개선 시 효과적
            weights['genre'] += 0.05
            weights['keywords'] += 0.05
            weights['kobert'] -= 0.07 # KoBERT 낮춤
            weights['overview_tfidf'] -= 0.08 # TF-IDF 낮춤

        # 3. 장르 수에 따른 조정 (if-elif-else 구조 사용)
        genre_count = len(movie_info['genres'].split(',')) if movie_info['genres'] else 0

        if genre_count == 1:  # 단일 장르: 해당 장르의 특성을 강하게 반영
            weights['genre'] += 0.10
            weights['overview_tfidf'] -= 0.05
            weights['keywords'] -= 0.05
            weights['kobert'] -= 0.02
        elif genre_count > 3:  # 많은 장르: 특정 장르에 덜 치우치도록
            weights['genre'] -= 0.03
            weights['kobert'] += 0.03 # KoBERT 영향력을 다시 높여 전체 줄거리 문맥 반영
        # else: (2개의 장르인 경우 기본 가중치 유지)

        # 4. 특정 장르에 대한 조정 (가장 구체적인 조건부터 배치)
        genres = movie_info['genres'].lower() if movie_info['genres'] else ''

        # 스릴러와 드라마 조합 (기생충, 하녀 등)
        if '드라마' in genres and '스릴러' in genres and '코미디' in genres: # 기생충 같은 경우
            weights['genre'] += 0.15 # 장르 매우 중요
            weights['keywords'] += 0.05
            weights['kobert'] -= 0.05 # KoBERT 비중 감소
        elif '드라마' in genres and '스릴러' in genres: # 일반적인 드라마/스릴러
            weights['genre'] += 0.10
            weights['keywords'] += 0.05
            weights['overview_tfidf'] += 0.02 # 줄거리 핵심 단어 중요도도 높임
        elif 'sf' in genres or '공포' in genres: # '괴물' 같은 경우
            weights['genre'] += 0.15 # 장르 매우 중요
            weights['keywords'] += 0.10 # SF/공포는 키워드가 중요 (괴물, 좀비 등)
            weights['overview_tfidf'] += 0.05 # 줄거리 내 핵심 단어 중요
            weights['kobert'] -= 0.10 # KoBERT 비중 대폭 감소
        elif '액션' in genres:
            weights['keywords'] += 0.05
            weights['overview_tfidf'] += 0.05 # 액션은 줄거리 내 동작 묘사가 중요
            weights['title'] -= 0.05 # 제목보다는 내용이 중요
        elif '로맨스' in genres:
            weights['kobert'] += 0.03 # 감성적인 줄거리 유사성 중요
            weights['overview_tfidf'] += 0.03
            weights['keywords'] -= 0.03
            weights['genre'] -= 0.03 # 로맨스는 장르가 넓어질 수 있으므로
        elif '코미디' in genres:
            weights['kobert'] += 0.02 # 코미디도 줄거리의 뉘앙스가 중요
            weights['genre'] += 0.05 # 코미디 장르의 특색
            weights['keywords'] -= 0.02

        # 음수 가중치 방지 및 최소값 설정 (선택 사항)
        for key in weights:
            if weights[key] < 0.01: # 최소 가중치 설정 (예: 1%)
                weights[key] = 0.01

        # 가중치 정규화
        total_weight = sum(weights.values())
        weights = {k: v/total_weight for k, v in weights.items()}
        
        return weights
    
    def recommend_movies(self, movie_title=None, movie_idx=None, top_n=10, weights=None, use_adaptive_weights=True):
        """영화 추천"""
        
        if movie_idx is None:
            if movie_title is None:
                raise ValueError("movie_title 또는 movie_idx 중 하나는 제공되어야 합니다.")
            
            # 제목으로 인덱스 찾기
            matches = self.movie_data[self.movie_data['title'].str.contains(movie_title, case=False, na=False)]
            if matches.empty:
                print(f"'{movie_title}'과 일치하는 영화를 찾을 수 없습니다.")
                return None
            
            movie_idx = matches.index[0]
            actual_title = matches.iloc[0]['title']
        else:
            actual_title = self.movie_data.iloc[movie_idx]['title']
        
        # 가중치 결정
        if use_adaptive_weights:
            weights = self.adaptive_weights(movie_idx)
            print(f"적응적 가중치 사용: {weights}")
        elif weights is None:
            weights = {
                'kobert': 0.15,
                'overview_tfidf': 0.35,
                'title': 0.10,
                'keywords': 0.25,
                'genre': 0.15
            }
        
        print(f"\n기준 영화: {actual_title}")
        print(f"장르: {self.movie_data.iloc[movie_idx]['genres']}")
        print(f"줄거리: {self.movie_data.iloc[movie_idx]['overview'][:400]}...")
        print(f"   키워드: {self.movie_data.iloc[movie_idx]['keywords']}")
        
        # 유사도 계산
        similarities = self.calculate_similarity_comprehensive(movie_idx, weights)
        final_similarities = similarities[0]
        
        # 자기 자신 제외하고 상위 N개 추천
        similar_indices = np.argsort(final_similarities)[::-1][1:top_n+1]
        
        # 추천 결과 생성
        recommendations = []
        for idx in similar_indices:
            movie_info = {
                'title': self.movie_data.iloc[idx]['title'],
                'genres': self.movie_data.iloc[idx]['genres'],
                'overview': self.movie_data.iloc[idx]['overview'],
                'keywords': self.movie_data.iloc[idx]['keywords'],
                'total_similarity': final_similarities[idx],
                'kobert_similarity': similarities[1][idx],
                'overview_tfidf_similarity': similarities[2][idx],
                'title_similarity': similarities[3][idx],
                'keyword_similarity': similarities[4][idx],
                'genre_similarity': similarities[5][idx]
            }
            recommendations.append(movie_info)
        
        return recommendations
    
    def display_recommendations(self, recommendations):
        """추천 결과 출력"""
        if not recommendations:
            print("추천할 영화가 없습니다.")
            return
        
        print(f"\n=== 추천 영화 TOP {len(recommendations)} ===")
        
        for i, movie in enumerate(recommendations, 1):
            print(f"\n{i}. {movie['title']}")
            print(f"   장르: {movie['genres']}")
            print(f"   총 유사도: {movie['total_similarity']:.3f}")
            print(f"   세부 유사도:")
            print(f"     - KoBERT: {movie['kobert_similarity']:.3f}")
            print(f"     - 줄거리 TF-IDF: {movie['overview_tfidf_similarity']:.3f}")
            print(f"     - 제목: {movie['title_similarity']:.3f}")
            print(f"     - 키워드: {movie['keyword_similarity']:.3f}")
            print(f"     - 장르: {movie['genre_similarity']:.3f}")
            print(f"   키워드: {movie['keywords']}")
            print(f"   줄거리: {movie['overview'][:100]}...")

In [29]:
# 실행 예시
if __name__ == "__main__":
    # 확장된 샘플 데이터 생성
    movies_df = pd.read_csv('../data_processing/content_data.csv', encoding='utf-8-sig')
    
    # 개선된 추천 시스템 초기화
    recommender = ImprovedMovieRecommendationSystem(cache_dir="./cached_features")
    
    # 데이터 전처리
    recommender.prepare_data(movies_df)
    
    # 적응적 가중치로 추천
    print("=== 어바웃 타임 기반 추천 (적응적 가중치) ===")
    recommendations1 = recommender.recommend_movies(
        movie_title='유희왕 듀얼몬스터즈',
        top_n=8,
        use_adaptive_weights=True
    )
    recommender.display_recommendations(recommendations1)

불용어 파일 로드 완료: ./stopwords/git_stopwords.txt (595개)
데이터 전처리 중...
✅ 저장된 특성 데이터를 불러옵니다...
✅ 모든 특성 데이터 로드 완료.
=== 어바웃 타임 기반 추천 (적응적 가중치) ===
적응적 가중치 사용: {'kobert': 0.1739130434782609, 'overview_tfidf': 0.34782608695652173, 'title': 0.04347826086956522, 'keywords': 0.2173913043478261, 'genre': 0.2173913043478261}

기준 영화: 유희왕 듀얼몬스터즈
장르: 애니메이션, 액션&모험
줄거리: 게임의 역사, 그것은 지금으로부터 5천년 전, 고대 이집트까지 거슬러 올라간다.

고대에 행해진 게임은 인간이나 왕의 미래를 예언하고 운명을 결정하는 마술적인 의식이었다. 그것들은 '어둠의 게임'이라 불렀다.

그리고, 지금 천년 퍼즐을 풀고 어둠의 게임을 계승한 소년이 있었으니...

빛과 어둠, 두 개의 마음을 가진 소년. 사람들은 그를 유희왕이라 부른다....
   키워드: ['게임', '어둠', '지금', '고대', '소년']

=== 추천 영화 TOP 8 ===

1. 소드 아트 온라인
   장르: SF&판타지, 애니메이션, 액션&모험
   총 유사도: 0.521
   세부 유사도:
     - KoBERT: 0.810
     - 줄거리 TF-IDF: 0.672
     - 제목: 0.000
     - 키워드: 0.206
     - 장르: 0.467
   키워드: ['게임', '온라인', '플레이어', '가상', '소드']
   줄거리: 2022년. 인류는 마침내 완전한 가상 공간을 실현했다. 모든 게이머가 꿈꿔왔던 VRMMORPG (가상 대규모 온라인 롤 플레잉 게임)  "소드 아트 온라인" 이 정식 가동을 시작...

2. 주먹왕 랄프
   장르: 가족, 모험, 애니메이션, 코미디
   총 유사도: 0.481
   세부 유사도